# CODE

In [2]:
# ============================================================
# @title INSTALL – Infra Qwen3 + LangGraph + Tabular KB (SQLite-vec) + setup.yaml
# ============================================================
# %%capture
# LLM / Infra Qwen3 + vLLM (razonamiento + tool-calling)
!pip -q install "vllm>=0.9.0" "huggingface_hub>=0.23.0" "openai==1.108.2" "jedi>=0.16"
!pip -q install "transformers>=4.57.3,<5"
!pip -q install langchain langchain-core langchain-openai langgraph typing_extensions
!pip -q install "git+https://github.com/whynpc9/langchain-qwq-vllm.git"

# Pydantic 2.x + YAML para setup.yaml + frontend mínimo
!pip -q install "pydantic>=2.7.0" "pyyaml>=6.0"
!pip -q install streamlit

# SQLite / VDB tabular (para KBs tipo FAOSTAT_SQLITE / FAOSTAT_VECTOR)
!pip -q install sqlite-vec pandas numpy

import os
import pathlib
import sys
import time
import subprocess
import json
import socket
import urllib.request

import streamlit  # noqa: F401
import sqlite3    # stdlib
import yaml       # para leer setup.yaml

print("✅ Dependencias base instaladas:")
print("   - vLLM / Qwen wrapper / LangChain / LangGraph")
print("   - Pydantic 2.x + PyYAML (setup.yaml como panel de control)")
print("   - Streamlit")
print("   - sqlite-vec + pandas + numpy para KB tabular/vectorial sobre SQLite")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.4/948.4 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 MB 1.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.6/192.6 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 91.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 99.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 58.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.4 MB/s eta 0:00:00:00:0100:01
   ━

In [4]:
!ls -la

total 16
drwxr-xr-x 1 root root 4096 Dec  9 14:41 .
drwxr-xr-x 1 root root 4096 Feb  6 19:38 ..
drwxr-xr-x 4 root root 4096 Dec  9 14:41 .config
drwxr-xr-x 1 root root 4096 Dec  9 14:42 sample_data


In [ ]:
# ============================================================
# @title `agnostic_agent`
# ============================================================
!mkdir -p agnostic_agent

In [ ]:
#@title agnostic_agent/schemas.py
%%writefile agnostic_agent/schemas.py
from __future__ import annotations

"""
Esquemas internos del Agnostic Deep Agent 2026.

Aquí definimos el "lenguaje interno" del grafo:

- AnalyzerIntent      → resultado del sub-grafo ANALYZER.
- PlannerPlan         → plan de pasos del sub-grafo PLANNER.
- ValidationResult    → veredicto del sub-grafo VALIDATOR.
- MemoryContext       → contexto recuperado por MEMORY_READ.
- AgentState          → TypedDict compartido por todos los nodos del grafo.

NOTA:
- Estos modelos NO son el I/O externo; para eso está `communication.py`.
"""

from typing import Any, Dict, List, Optional, TypedDict

from pydantic import BaseModel, Field

from .communication import ToolRun, AgentSummary


# ─────────────────────────────────────────────
# ANALYZER
# ─────────────────────────────────────────────

class RequiredItem(BaseModel):
    """
    Ítem que DEBE quedar cubierto en la respuesta final.

    - id: identificador corto (ej. "q1", "step_b", ...).
    - description: texto que describe qué debe resolverse.
    - must_be_answered: si es obligatorio o sólo deseable.
    """
    id: str = Field(..., description="Identificador corto del ítem requerido.")
    description: str = Field(..., description="Descripción del ítem requerido.")
    must_be_answered: bool = Field(
        default=True,
        description="Si True, la respuesta final DEBE cubrir este ítem.",
    )


class AnalyzerIntent(BaseModel):
    """
    Resultado del ANALYZER:

    - logic_form: representación lógica (ej. 'q1 ∧ q2 ∧ (q3 ∨ q4)').
    - subqueries: lista de subconsultas / subtareas en texto.
    - required_items: lista de RequiredItem que SUMMARIZER debe cubrir.
    - wants_tool_trace: si el usuario quiere ver trazas técnicas.
    - language: idioma dominante del usuario (si se detecta).
    """
    logic_form: str = ""
    subqueries: List[str] = Field(default_factory=list)
    required_items: List[RequiredItem] = Field(default_factory=list)
    wants_tool_trace: bool = False
    language: Optional[str] = None


# ─────────────────────────────────────────────
# PLANNER
# ─────────────────────────────────────────────

class PlannedStep(BaseModel):
    """
    Un paso del plan del PLANNER.

    - id: identificador único del step (ej. 'step1').
    - tool_name: nombre de la tool a invocar.
    - args: argumentos (posiblemente con referencias tipo '$stepX.result').
    - depends_on: ids de otros steps de los que depende.
    """
    id: str
    tool_name: str
    args: Dict[str, Any] = Field(default_factory=dict)
    depends_on: List[str] = Field(default_factory=list)


class PlannerPlan(BaseModel):
    """
    Plan completo devuelto por el PLANNER como DAG de steps.
    """
    steps: List[PlannedStep] = Field(default_factory=list)
    notes: Optional[str] = None


# ─────────────────────────────────────────────
# VALIDATOR
# ─────────────────────────────────────────────

class ValidationResult(BaseModel):
    """
    Resultado del VALIDATOR:

    - all_covered: True si todos los RequiredItem se consideran cubiertos.
    - missing_item_ids: ids de RequiredItem que faltan por cubrir.
    - comments: texto adicional (por ejemplo, sugerencias al SUMMARIZER).
    """
    all_covered: bool
    missing_item_ids: List[str] = Field(default_factory=list)
    comments: Optional[str] = None


# ─────────────────────────────────────────────
# MEMORY
# ─────────────────────────────────────────────

class MemoryContext(BaseModel):
    """
    Contexto de memoria agregado por MEMORY_READ.

    - session_history: extractos de la conversación actual.
    - short_term_snippets: resúmenes o puntos clave recientes.
    - long_term_snippets: recuerdos persistentes relevantes (vector store, etc.).
    """
    session_history: List[str] = Field(default_factory=list)
    short_term_snippets: List[str] = Field(default_factory=list)
    long_term_snippets: List[str] = Field(default_factory=list)


# ─────────────────────────────────────────────
# STATE GLOBAL (LangGraph)
# ─────────────────────────────────────────────

class AgentState(TypedDict, total=False):
    """
    Estado compartido entre nodos del grafo.

    Claves más relevantes:
    - user_prompt: texto original del usuario (último turno).
    - session_id, user_id: identificadores lógicos.
    - setup_path: ruta al setup.yaml activo.
    - kb_names: nombres de KBs solicitadas por el usuario.

    - memory_context: MemoryContext completado por MEMORY_READ.
    - analyzer_intent: AnalyzerIntent generado por ANALYZER.
    - planner_plan: PlannerPlan generado por PLANNER.

    - step_results: resultados crudos por id de step.
    - tool_runs: lista de ToolRun (normalizados en CATCHER).
    - validation: ValidationResult del VALIDATOR.

    - pipeline_summary: AgentSummary opcional con resumen por fase.
    - user_out / deep_out / dev_out: textos finales por vista.
    """
    # Identificadores y config
    user_prompt: str
    session_id: str
    user_id: Optional[str]
    setup_path: str
    kb_names: List[str]

    # Contexto de memoria
    memory_context: MemoryContext

    # Resultados de sub-grafos
    analyzer_intent: AnalyzerIntent
    planner_plan: PlannerPlan
    validation: ValidationResult

    # Ejecución de tools
    step_results: Dict[str, Any]
    tool_runs: List[ToolRun]

    # Resúmenes y salidas finales
    pipeline_summary: AgentSummary
    user_out: str
    deep_out: str
    dev_out: str


In [ ]:
# @title agnostic_agent/memory.py
%%writefile agnostic_agent/memory.py
from __future__ import annotations

"""
Memoria del Agnostic Deep Agent 2026.

Diseño multi-nivel (versión mínima):

- Session memory:
    Historial reciente de la conversación (texto plano).
- Short-term memory:
    Snippets/resúmenes recientes cada N turnos.
- Long-term memory:
    Placeholder para recuerdos persistentes (vector store, SQL, etc.).

Este módulo NO depende de LangGraph directamente; sólo expone funciones
para leer/escribir memoria usando estructuras sencillas, en términos de:

- session_id: identificador de la conversación.
- MemoryContext: modelo Pydantic definido en schemas.py.
"""

from typing import Any, Dict, List, Optional
from datetime import datetime

from .schemas import MemoryContext


# ─────────────────────────────────────────────
# Almacenamiento in-memory (versión Colab / prototipo)
# ─────────────────────────────────────────────

# Historial de mensajes por sesión (texto plano)
#   _SESSION_STORE[session_id] = ["USER: ...", "AGENT: ...", ...]
_SESSION_STORE: Dict[str, List[str]] = {}

# Snippets/resúmenes de corto plazo por sesión
#   _SHORT_TERM_STORE[session_id] = ["resumen 1", "resumen 2", ...]
_SHORT_TERM_STORE: Dict[str, List[str]] = {}

# Long-term docs por usuario (placeholder; lo normal será un vector store)
#   _LONG_TERM_STORE[user_id] = [
#       {"text": "...", "created_at": "...", "meta": {...}},
#       ...
#   ]
_LONG_TERM_STORE: Dict[str, List[Dict[str, Any]]] = {}


# ─────────────────────────────────────────────
# Helpers internos
# ─────────────────────────────────────────────

def _now_iso() -> str:
    return datetime.utcnow().isoformat() + "Z"


def _truncate_list(items: List[Any], max_len: int) -> List[Any]:
    if max_len <= 0:
        return []
    if len(items) <= max_len:
        return items
    return items[-max_len:]


def _build_short_term_snippet(user_text: str, agent_text: str, turn_idx: int) -> str:
    """
    Snippet simple de corto plazo: pensado como placeholder.

    Puedes reemplazar esto más adelante por un LLM summarizer dedicado.
    """
    u = user_text.strip().replace("\n", " ")
    a = agent_text.strip().replace("\n", " ")
    if len(u) > 120:
        u = u[:117] + "..."
    if len(a) > 160:
        a = a[:157] + "..."
    return f"[turn {turn_idx}] USER: {u} | AGENT: {a}"


# ─────────────────────────────────────────────
# Lectura de memoria
# ─────────────────────────────────────────────

def read_memory(
    session_id: str,
    *,
    max_session_messages: int = 50,
    max_short_snippets: int = 20,
) -> MemoryContext:
    """
    Construye un MemoryContext para una sesión dada.

    - max_session_messages: cuántas líneas de historial traer (USER/AGENT mezcladas).
    - max_short_snippets:  cuántos snippets de short-term traer.
    """
    session_msgs = _SESSION_STORE.get(session_id, [])
    short_snips = _SHORT_TERM_STORE.get(session_id, [])

    session_history = _truncate_list(session_msgs, max_session_messages)
    short_term_snippets = _truncate_list(short_snips, max_short_snippets)

    # Long-term: de momento no filtramos por relevancia aquí; eso se hará
    # en otro módulo (vector store, RAG, etc.). Por ahora sólo devolvemos textos.
    long_term_texts: List[str] = []
    # Nota: la clave para LONG_TERM_STORE es user_id, no session_id;
    # aquí no lo conocemos, así que devolvemos lista vacía.
    # Más adelante se puede ampliar la firma para incluir user_id.

    return MemoryContext(
        session_history=session_history,
        short_term_snippets=short_term_snippets,
        long_term_snippets=long_term_texts,
    )


# ─────────────────────────────────────────────
# Escritura / actualización de memoria
# ─────────────────────────────────────────────

def write_memory(
    session_id: str,
    *,
    user_prompt: str,
    user_out: str,
    user_id: Optional[str] = None,
    memory_cfg: Optional[Dict[str, Any]] = None,
) -> None:
    """
    Actualiza las memorias tras un turno completado.

    Parámetros:
    - session_id: id de sesión.
    - user_prompt: último mensaje del usuario.
    - user_out: respuesta final del agente para ese turno.
    - user_id: id lógico del usuario (para long-term).
    - memory_cfg: configuración opcional con la forma:

        {
          "session": {
            "enabled": true,
            "max_messages": 50,
          },
          "short_term": {
            "enabled": true,
            "summarize_every_n_turns": 5,
            "window_turns": 20,
          },
          "long_term": {
            "enabled": false,
            "max_docs": 200,
          }
        }

      Si algún bloque o campo no está presente, se usan valores por defecto.
    """
    cfg = memory_cfg or {}

    sess_cfg = cfg.get("session", {}) or {}
    st_cfg = cfg.get("short_term", {}) or {}
    lt_cfg = cfg.get("long_term", {}) or {}

    session_enabled: bool = bool(sess_cfg.get("enabled", True))
    max_messages: int = int(sess_cfg.get("max_messages", 50))

    short_enabled: bool = bool(st_cfg.get("enabled", True))
    summarize_every_n_turns: int = int(st_cfg.get("summarize_every_n_turns", 5))
    window_turns: int = int(st_cfg.get("window_turns", 20))

    long_enabled: bool = bool(lt_cfg.get("enabled", False))
    long_max_docs: int = int(lt_cfg.get("max_docs", 200))

    # -------------------------
    # 1) Session memory
    # -------------------------
    if session_enabled:
        msgs = _SESSION_STORE.setdefault(session_id, [])
        msgs.append(f"USER: {user_prompt}")
        msgs.append(f"AGENT: {user_out}")
        # Recortamos a las últimas `max_messages` líneas
        _SESSION_STORE[session_id] = _truncate_list(msgs, max_messages)

    # Número de turnos aproximado (USER/AGENT cuentan como dos mensajes)
    num_turns = len(_SESSION_STORE.get(session_id, [])) // 2 if session_enabled else 0

    # -------------------------
    # 2) Short-term memory
    # -------------------------
    if short_enabled and num_turns > 0:
        # Cada N turnos, generamos un nuevo snippet
        if summarize_every_n_turns > 0 and num_turns % summarize_every_n_turns == 0:
            snippet = _build_short_term_snippet(user_prompt, user_out, num_turns)
            st_list = _SHORT_TERM_STORE.setdefault(session_id, [])
            st_list.append(snippet)
            # Recortamos por ventana de turnos (como proxy)
            _SHORT_TERM_STORE[session_id] = _truncate_list(st_list, window_turns)

    # -------------------------
    # 3) Long-term memory (placeholder)
    # -------------------------
    if long_enabled and user_id:
        docs = _LONG_TERM_STORE.setdefault(user_id, [])
        # Heurística mínima: guardar sólo turnos cuyo user_out tenga cierta longitud
        if len(user_out.strip()) >= 40:
            docs.append(
                {
                    "text": f"USER: {user_prompt.strip()}\nAGENT: {user_out.strip()}",
                    "created_at": _now_iso(),
                    "meta": {
                        "session_id": session_id,
                        "turn_index": num_turns,
                    },
                }
            )
            _LONG_TERM_STORE[user_id] = _truncate_list(docs, long_max_docs)


# ─────────────────────────────────────────────
# Utilidades varias
# ─────────────────────────────────────────────

def clear_session_memory(session_id: str) -> None:
    """
    Elimina TODA la memoria asociada a una sesión concreta
    (session history + short-term snippets).
    """
    _SESSION_STORE.pop(session_id, None)
    _SHORT_TERM_STORE.pop(session_id, None)


def clear_long_term_memory(user_id: str) -> None:
    """
    Elimina la memoria de largo plazo de un usuario concreto.
    """
    _LONG_TERM_STORE.pop(user_id, None)


def debug_dump_memory() -> Dict[str, Any]:
    """
    Devuelve una vista simple de todas las memorias en memoria (para depuración).
    NO usar en producción si el contenido es sensible.
    """
    return {
        "session_store": dict(_SESSION_STORE),
        "short_term_store": dict(_SHORT_TERM_STORE),
        "long_term_store": dict(_LONG_TERM_STORE),
    }


In [ ]:
#@title agnostic_agent/prompts.py
%%writefile agnostic_agent/prompts.py
from __future__ import annotations

"""
Prompts de sistema para el Agnostic Deep Agent 2026.

Aquí centralizamos TODO el comportamiento de alto nivel en texto:

- ANALYZER_SYSTEM_PROMPT
- SUMMARIZER_*_SYSTEM_PROMPT  (user / deep / dev)
- VALIDATOR_SYSTEM_PROMPT
- MEMORY_WRITE_SYSTEM_PROMPT  (decidir qué guardar a largo plazo)

NOTA:
- Estos prompts son agnósticos de dominio (no asumen FAOSTAT, banca, etc.),
  pero están pensados para trabajar con Knowledge Bases (KBs) y tablas
  estructuradas (CSV/SQL) como contexto adicional.
- El wiring con modelos (SystemMessage, etc.) se hace fuera (p.ej. en
  capabilities.py o logic.py). Aquí solo definimos textos y helpers ligeros.
"""

from typing import Literal

from langchain_core.messages import SystemMessage


# ─────────────────────────────────────────────
# ANALYZER – de texto libre a AnalyzerIntent
# ─────────────────────────────────────────────

ANALYZER_SYSTEM_PROMPT: str = """
Eres el ANALYZER de un agente de IA de propósito general.

Tu trabajo es LEER con cuidado la petición del usuario y devolver un
objeto JSON que represente su intención de forma estructurada.

La entrada que recibirás incluye, como mínimo:
- user_prompt: texto completo del usuario.
- memory_context: resúmenes y fragmentos de contexto previos (si existen).

Y el sistema puede disponer además de:
- knowledge_bases: descriptores de BDs/tablas/KBs disponibles
  (por ejemplo, tablas SQL, vectores, diccionarios de negocio).
- context_tables: rutas a tablas CSV de contexto (por ejemplo,
  tablas de parametrías, diccionarios de abreviaturas/definiciones,
  catálogos de atributos, etc.).

NO debes devolver estos campos en el JSON, pero sí debes tener en mente
que parte del problema puede requerir:
- cruzar una "tabla de atributos" (input A) con una o varias tablas
  de contexto (input B) como parametrías o diccionarios;
- aplicar reglas de negocio, abreviaturas o definiciones que residen
  en esas tablas.

DEBES devolver UN ÚNICO objeto JSON con esta estructura EXACTA:

{
  "logic_form": "<cadena que represente la lógica proposicional de las subconsultas>",
  "subqueries": [
    "<subconsulta 1 en texto>",
    "<subconsulta 2 en texto>",
    ...
  ],
  "required_items": [
    {
      "id": "<id_corto_ej_q1>",
      "description": "<qué debe responderse en lenguaje natural>",
      "must_be_answered": true
    },
    ...
  ],
  "wants_tool_trace": true,
  "language": "<idioma_dominante_ej_es_o_en>"
}

Instrucciones:

1) Descompón el mensaje en subconsultas claras y numeradas cuando tenga
   varias partes (por ejemplo: "primero haz A, luego B...").

   - Si el usuario habla de una TABLA de atributos A y una TABLA de
     contexto B (parametrías, abreviaturas, diccionarios), refleja eso
     en las subqueries y en los required_items, indicando qué juicio o
     salida se espera a partir de ese cruce.

2) Para cada subconsulta importante, crea un RequiredItem con:
   - id: "q1", "q2", "q3", etc.
   - description: qué espera exactamente el usuario como respuesta,
     incluso si la respuesta se obtendrá aplicando reglas sobre tablas.
   - must_be_answered: true si es obligatorio, false si es opcional.

3) logic_form puede usar conectores lógicos simples:
   - "q1 ∧ q2", "q1 ∧ (q2 ∨ q3)", etc.
   - Si hay dependencias (por ejemplo: primero identificar el contrato,
     luego aplicar parametrías), refleja esa estructura en logic_form.

4) wants_tool_trace:
   - true si el usuario pide explícitamente ver "cómo razonaste",
     "qué herramientas usaste", "explica el proceso", etc.
   - false en caso contrario.

5) language:
   - "es" si el usuario escribe principalmente en español,
   - "en" si escribe en inglés,
   - otro código ISO simple si detectas otro idioma.

6) No añadas comentarios fuera del JSON. Devuelve SOLO el JSON.
""".strip()


def build_analyzer_system_message() -> SystemMessage:
    """Devuelve el SystemMessage para el rol ANALYZER."""
    return SystemMessage(content=ANALYZER_SYSTEM_PROMPT)


# ─────────────────────────────────────────────
# SUMMARIZER – user / deep / dev
# ─────────────────────────────────────────────

SUMMARIZER_USER_SYSTEM_PROMPT: str = """
Eres el SUMMARIZER (vista USUARIO) de un agente de IA.

Recibirás:
- analyzer_intent: con required_items y lógica.
- tool_runs: lista de ejecuciones de tools (ya normalizadas).
- step_results: resultados por id de step.
- memory_context: contexto de la conversación.
- hints del VALIDADOR (missing_items), si se trata de un reintento.

El sistema puede haber consultado:
- Knowledge Bases (KBs) tabulares o vectoriales.
- Tablas de contexto (parametrías, diccionarios de abreviaturas, etc.).
- Documentos que simulan OCR de contratos u otros textos.

Tu objetivo es producir UNA ÚNICA respuesta en lenguaje natural para el usuario:

- Clara, breve y orientada a la acción.
- En el mismo idioma que el usuario (campo analyzer_intent.language).
- Cubriendo TODOS los required_items marcados como must_be_answered=true.
- Si analyzer_intent.wants_tool_trace es true, incluye una sección breve
  explicando qué se hizo (sin entrar en detalles técnicos extremos).

Instrucciones:

1) Empieza por responder directamente a la petición principal.

   - Si la respuesta depende de cruzar una fila de atributos (ej. número
     de contrato, tipo de operación, etc.) con tablas de parametrías o
     abreviaturas, deja claro que tu juicio se basa en esas reglas
     y diccionarios, NO en opiniones arbitrarias.

2) Asegúrate de cubrir cada RequiredItem obligatorio (puedes usar viñetas).

3) Si hay errores de alguna tool (por ejemplo, fallo al leer una tabla,
   problemas en embeddings o en búsquedas), explícalos de forma amable y
   propone alternativas o aclaraciones.

4) Si el usuario pide trazas (wants_tool_trace=true), añade al final una
   sección breve tipo:
   - "Resumen del proceso" → indicando:
     - qué tablas / KBs se consultaron,
     - qué tipo de matching se hizo (semántico, exacto, etc.),
     - y cómo se aplicaron las reglas o definiciones.

5) No incluyas el JSON interno ni IDs de pasos a menos que el usuario pida
   explícitamente detalles técnicos.

6) No agregues nada fuera del texto final dirigido al usuario.
""".strip()


SUMMARIZER_DEEP_SYSTEM_PROMPT: str = """
Eres el SUMMARIZER (vista DEEP) de un agente de IA.

Tu audiencia es una persona TÉCNICA que quiere entender qué hizo el agente,
no sólo ver la respuesta final.

Recibirás:
- analyzer_intent (logic_form, subqueries, required_items, etc.).
- planner_plan (si existe).
- tool_runs normalizados.
- step_results.
- memory_context (fragmentos relevantes).
- Información indirecta sobre qué KBs y tablas de contexto se usaron
  (por ejemplo, semantic_search_in_csv, consultas SQL, vector search, etc.).

Debes devolver un texto en formato markdown, estructurado más o menos así:

## Resumen de alto nivel
(una o dos frases sobre qué se hizo)

### ANALYZER
- Lógica proposicional: ...
- Subconsultas detectadas: ...
- Required items: ...
- Relación entre input A (atributos/tablas) e input B (tablas de contexto)
  si aplica (por ejemplo: "fila de contrato vs. tablas de parametrías y
  diccionario de abreviaturas").

### PLANNER
- Descripción general del plan
- Pasos planificados (en orden lógico)
- Cómo se decidió usar ciertas KBs / tablas de contexto (si se ve en el plan).

### EXECUTOR
- Lista de tools efectivamente llamadas y para qué se usaron.
  - Incluye, cuando existan:
    - búsquedas semánticas en CSV,
    - consultas SQL a KBs tabulares,
    - re-rankers o embeddings aplicados sobre documentos (ej. OCR).
- Comentarios sobre errores o reintentos, si los hubo.
- Explica cómo se cruzó la información de:
  - registros tabulares (input A),
  - tablas de parametrías / diccionarios (context_tables),
  - y documentos de texto (OCR, contratos, etc.).

### CATCHER
- Notas sobre normalización / truncado / saneamiento (si aplica).

### SUMMARIZER
- Cómo se construyó la respuesta final para el usuario,
  incluyendo cómo se tradujeron las reglas/tablas a lenguaje natural.

### Respuesta final (resumen)
- Pequeño resumen de lo que recibió el usuario (sin repetirlo completo).

Instrucciones:
- Usa un tono técnico pero legible.
- No vuelvas a listar datos gigantes (listas enormes, matrices…); sólo
  describe su rol o muestra pequeños extractos representativos.
- NO devuelvas JSON; solo markdown.
""".strip()


SUMMARIZER_DEV_SYSTEM_PROMPT: str = """
Eres el SUMMARIZER (vista DEV) de un agente de IA.

Tu audiencia son desarrolladores que quieren depurar o auditar el comportamiento.

Recibirás:
- analyzer_intent
- planner_plan
- tool_runs
- step_results
- memory_context
- fragmentos del estado crudo del grafo (si el llamador lo incluye)

Debes devolver un texto en formato markdown con énfasis en:

- IDs de steps, tools y tiempo de ejecución (si se proveen).
- Inputs y outputs relevantes (resumen de payloads grandes).
- Errores, excepciones o casos no cubiertos.
- Cualquier inconsistencia detectada.
- Uso concreto de KBs / tablas de contexto (qué backend y qué tabla se
  consultó: SQLite, sqlite-vec, CSV, etc., según se vea en los tool_runs).

Estructura sugerida:

## DEV TRACE (alto nivel)
- Descripción breve del turno (qué se intentó hacer).

## ANALYZER
- Payload relevante (resumen).
- Cómo se mapearon las partes del prompt a required_items (q1, q2, ...).

## PLANNER
- Plan final (steps, depends_on).
- Decisiones relevantes (ej. "primero localizar contrato, luego aplicar
  parametrías y validar abreviaturas").

## EXECUTOR / TOOLS
- Tabla o lista de tool_runs con:
  - step_id / tool_name
  - args relevantes (truncados)
  - KB / backend implicado (si es claro: csv, sqlite, sqlite-vec, etc.)
  - tipo de salida (embedding, texto, número, filas tabulares, etc.)
  - errores (si los hubo) y cómo se gestionaron.

## STATE SNAPSHOT
- Notas sobre campos importantes del estado (state), por ejemplo:
  - kb_selected, context_tables, context_cfg, flags de validación, etc.

No incluyas credenciales, PII o datos sensibles si aparecen en el estado.
Trúncalos o marca que fueron redacted.
""".strip()


def build_summarizer_system_message(
    view: Literal["user", "deep", "dev"] = "user",
) -> SystemMessage:
    """
    Devuelve el SystemMessage adecuado para el SUMMARIZER según vista.
    """
    if view == "deep":
        return SystemMessage(content=SUMMARIZER_DEEP_SYSTEM_PROMPT)
    if view == "dev":
        return SystemMessage(content=SUMMARIZER_DEV_SYSTEM_PROMPT)
    # por defecto, vista usuario
    return SystemMessage(content=SUMMARIZER_USER_SYSTEM_PROMPT)


# ─────────────────────────────────────────────
# VALIDATOR – comprobar RequiredItems vs respuesta
# ─────────────────────────────────────────────

VALIDATOR_SYSTEM_PROMPT: str = """
Eres el VALIDATOR de un agente de IA.

Tu trabajo es revisar si la respuesta final generada para el usuario
cubre TODOS los ítems requeridos (RequiredItem) proporcionados por el ANALYZER.

Recibirás:
- required_items: lista de objetos con campos {id, description, must_be_answered}.
- draft_answer: texto de la respuesta propuesta para el usuario.

Debes devolver UN ÚNICO objeto JSON con esta forma EXACTA:

{
  "all_covered": true,
  "missing_item_ids": ["q2", "q3"],
  "comments": "Texto libre opcional."
}

Reglas:

1) all_covered:
   - true si TODOS los RequiredItem con must_be_answered=true
     están razonablemente cubiertos en draft_answer.
   - false en caso contrario.

   Ten en cuenta que algunas descriptions pueden hacer referencia a
   resultados de aplicar reglas de negocio sobre tablas (parametrías,
   diccionarios, etc.). No necesitas conocer las tablas; solo verificar
   que el draft_answer responde a lo que se describe en cada item.

2) missing_item_ids:
   - lista de los ids de RequiredItem que consideres que NO están
     bien cubiertos (sólo los que tienen must_be_answered=true).

3) comments:
   - texto opcional explicando por qué falta algo, o sugerencias
     de cómo mejorar la respuesta.

4) No añadas nada fuera del JSON. Devuelve SOLO el JSON.
""".strip()


def build_validator_system_message() -> SystemMessage:
    """Devuelve el SystemMessage para el rol VALIDATOR."""
    return SystemMessage(content=VALIDATOR_SYSTEM_PROMPT)


# ─────────────────────────────────────────────
# MEMORY_WRITE – decidir qué guardar a largo plazo
# ─────────────────────────────────────────────

MEMORY_WRITE_SYSTEM_PROMPT: str = """
Eres el módulo de decisión de MEMORIA DE LARGO PLAZO de un agente de IA.

Tu tarea es decidir si la interacción actual merece ser almacenada como
recuerdo persistente (long-term memory).

Recibirás:
- user_prompt: mensaje del usuario.
- user_out: respuesta final del agente.
- metadata opcional (ej. etiquetas, importancia, etc.).

Debes devolver UN ÚNICO objeto JSON con la forma:

{
  "should_store": true,
  "summary": "Resumen breve del conocimiento o preferencia a guardar.",
  "tags": ["preferencia", "definicion", "dato_importante"]
}

Criterios para should_store:

- true si:
  - el usuario revela una preferencia estable (gustos, estilo, idioma),
  - se define una regla que se usará en el futuro (por ejemplo,
    una nueva parametría o criterio de evaluación para contratos),
  - se captura un conocimiento útil que probablemente se reutilice
    (por ejemplo, cómo interpretar cierto atributo tabular específico).

- false si:
  - es una pregunta puntual sin relevancia futura,
  - es información obsoleta o muy específica de un contexto efímero.

summary:
- una o dos frases como máximo.
- NO repitas el diálogo entero; sólo el conocimiento clave.

tags:
- lista corta de etiquetas en minúsculas (ej. ["preferencia", "contratos"],
  ["regla", "parametrias"], ["kb", "tabular"]).

No añadas nada fuera del JSON. Devuelve SOLO el JSON.
""".strip()


def build_memory_write_system_message() -> SystemMessage:
    """Devuelve el SystemMessage para el rol MEMORY_WRITE."""
    return SystemMessage(content=MEMORY_WRITE_SYSTEM_PROMPT)


In [ ]:
#@title agnostic_agent/capabilities.py
%%writefile agnostic_agent/capabilities.py
from __future__ import annotations

"""
Infraestructura de *capacidades* (modelos/servidores) para el Agnostic Deep Agent 2026.

Incluye:
- Descarga de modelos Qwen3 (LLM / Embeddings / Reranker) desde Hugging Face.
- Lanzamiento de servidores vLLM OpenAI-compatible (LLM / EMB / RERANK).
- Configuración del planner (PlannerConfig) y construcción del LLM planner
  (build_planner_llm + build_planner_system_message) sobre ChatQwenVllm.

NOTA:
- Esta lógica es agnóstica del dominio; sólo gestiona modelos y servidores.
- El wiring con LangGraph y el resto del agente se hace en logic.py y agent.py.
"""

from dataclasses import dataclass
from typing import Optional, Literal
import os
import sys
import subprocess
import time
import socket
import json
import urllib.request
import pathlib

from huggingface_hub import snapshot_download
from langchain_core.messages import SystemMessage
from langchain_qwq import ChatQwenVllm


# ─────────────────────────────────────────────
# Helpers básicos
# ─────────────────────────────────────────────

def _str_to_bool(x: str) -> bool:
    return str(x).lower() in ("1", "true", "yes", "y", "on")


# ─────────────────────────────────────────────
# Descarga de modelos Qwen3 (HF snapshot)
# ─────────────────────────────────────────────

@dataclass
class QwenModelPaths:
    """Rutas locales a los modelos Qwen usados por el agente."""
    llm_dir: str
    emb_dir: str
    rerank_dir: str


def ensure_model_downloaded(model_id: str, local_dir: pathlib.Path) -> str:
    """
    Descarga el modelo de Hugging Face a `local_dir` si no existe.
    Devuelve la ruta absoluta como string.
    """
    local_dir = pathlib.Path(local_dir)
    if local_dir.is_dir():
        return str(local_dir.resolve())

    local_dir.parent.mkdir(parents=True, exist_ok=True)
    snapshot_download(
        repo_id=model_id,
        local_dir=str(local_dir),
        local_dir_use_symlinks=False,
        ignore_patterns=["*.md", "*.png", "*.jpg", "LICENSE*"],
    )
    return str(local_dir.resolve())


def prepare_qwen_models(
    llm_model_id: Optional[str] = None,
    emb_model_id: Optional[str] = None,
    rerank_model_id: Optional[str] = None,
    base_dir: str | os.PathLike = "LM_MODEL",
) -> QwenModelPaths:
    """
    Descarga (si es necesario) los modelos Qwen y devuelve sus rutas locales.

    Si algún ID no se pasa, se lee de las variables de entorno o se usan defaults:
    - LLM_MODEL_ID      (por defecto: Qwen/Qwen3-0.6B)
    - EMB_MODEL_ID      (por defecto: Qwen/Qwen3-Embedding-0.6B)
    - RERANK_MODEL_ID   (por defecto: Qwen/Qwen3-Reranker-0.6B)
    """
    base_dir_path = pathlib.Path(base_dir)
    base_dir_path.mkdir(parents=True, exist_ok=True)

    llm_model_id = llm_model_id or os.getenv("LLM_MODEL_ID", "Qwen/Qwen3-0.6B")
    emb_model_id = emb_model_id or os.getenv("EMB_MODEL_ID", "Qwen/Qwen3-Embedding-0.6B")
    rerank_model_id = rerank_model_id or os.getenv("RERANK_MODEL_ID", "Qwen/Qwen3-Reranker-0.6B")

    llm_dir = ensure_model_downloaded(llm_model_id, base_dir_path / "Qwen3_LLM_MAIN")
    emb_dir = ensure_model_downloaded(emb_model_id, base_dir_path / "Qwen3_Embedding_0.6B")
    rerank_dir = ensure_model_downloaded(rerank_model_id, base_dir_path / "Qwen3_Reranker_0.6B")

    return QwenModelPaths(llm_dir=llm_dir, emb_dir=emb_dir, rerank_dir=rerank_dir)


# ─────────────────────────────────────────────
# Servidores vLLM (OpenAI-compatible)
# ─────────────────────────────────────────────

@dataclass
class VllmConfig:
    # Host / puertos
    host: str = "127.0.0.1"
    llm_port: int = int(os.getenv("LLM_PORT", "8000"))
    emb_port: int = int(os.getenv("EMB_PORT", "8001"))
    rerank_port: int = int(os.getenv("RERANK_PORT", "8002"))

    # Presupuesto de VRAM por servidor
    llm_gpu_util: float = float(os.getenv("VLLM_LLM_GPU_UTIL", "0.4"))
    emb_gpu_util: float = float(os.getenv("VLLM_EMB_GPU_UTIL", "0.3"))
    rerank_gpu_util: float = float(os.getenv("VLLM_RERANK_GPU_UTIL", "0.3"))

    # Longitud máxima de contexto por servidor
    llm_max_len: int = int(os.getenv("VLLM_LLM_MAX_LEN", "2048"))
    emb_max_len: int = int(os.getenv("VLLM_EMB_MAX_LEN", "1024"))
    rerank_max_len: int = int(os.getenv("VLLM_RERANK_MAX_LEN", "1024"))

    # Concurrencia (nº de secuencias simultáneas)
    llm_max_num_seqs: int = int(os.getenv("VLLM_LLM_MAX_NUM_SEQS", "4"))
    emb_max_num_seqs: int = int(os.getenv("VLLM_EMB_MAX_NUM_SEQS", "4"))
    rerank_max_num_seqs: int = int(os.getenv("VLLM_RERANK_MAX_NUM_SEQS", "4"))

    # Nombres "served_model_name" dentro de vLLM
    llm_served_name: str = os.getenv("LLM_SERVED_NAME", "qwen3-0.6b")
    emb_served_name: str = os.getenv("EMB_SERVED_NAME", "qwen3-embedding-0.6b")
    rerank_served_name: str = os.getenv("RERANK_SERVED_NAME", "qwen3-reranker-0.6b")

    # Parser de tool-calls y razonamiento (alineado Qwen3 + langchain-qwq)
    # Por defecto usamos el parser XML de Qwen3, que es el que mejor encaja con langchain-qwq.
    tool_call_parser: str = os.getenv("VLLM_TOOL_CALL_PARSER", "qwen3_xml")
    enable_reasoning: bool = _str_to_bool(os.getenv("VLLM_ENABLE_REASONING", "1"))
    reasoning_parser: Optional[str] = os.getenv("VLLM_REASONING_PARSER", "qwen3")

    # Flags para arrancar (o no) servidores extra
    start_emb_server: bool = _str_to_bool(os.getenv("VLLM_START_EMB_SERVER", "0"))
    start_rerank_server: bool = _str_to_bool(os.getenv("VLLM_START_RERANK_SERVER", "0"))


@dataclass
class VllmServers:
    llm_proc: subprocess.Popen
    emb_proc: Optional[subprocess.Popen] = None
    rerank_proc: Optional[subprocess.Popen] = None
    llm_log_path: str = ""
    emb_log_path: Optional[str] = None
    rerank_log_path: Optional[str] = None


@dataclass
class VllmEndpoints:
    llm_base_url: str
    emb_base_url: Optional[str] = None
    rerank_base_url: Optional[str] = None


def _free_port_if_needed(host: str, port: int) -> None:
    try:
        s = socket.socket()
        s.settimeout(0.5)
        if s.connect_ex((host, port)) == 0:
            _ = os.system(f"fuser -k {port}/tcp || true")
        s.close()
    except Exception:
        pass


def _url_open_no_proxy(url: str, timeout: float = 2.0) -> dict:
    opener = urllib.request.build_opener(urllib.request.ProxyHandler({}))
    urllib.request.install_opener(opener)
    with urllib.request.urlopen(url, timeout=timeout) as resp:
        return json.load(resp)


def _wait_until_ready(
    url: str,
    server_proc: subprocess.Popen,
    log_path: str,
    seconds: int = 180,
    sleep: float = 2.0,
) -> bool:
    start = time.time()
    while time.time() - start < seconds:
        if server_proc.poll() is not None:
            print(f"❌ Servidor terminó con código {server_proc.returncode}. Log tail:")
            try:
                with open(log_path, "r", encoding="utf-8", errors="ignore") as f:
                    lines = f.read().splitlines()
                print("\n".join(lines[-80:]))
            except Exception:
                pass
            return False
        try:
            _ = _url_open_no_proxy(url, timeout=2.5)
            return True
        except Exception:
            time.sleep(sleep)
    print("⏰ Timeout esperando servidor, mostrando tail del log:")
    try:
        with open(log_path, "r", encoding="utf-8", errors="ignore") as f:
            lines = f.read().splitlines()
        print("\n".join(lines[-80:]))
    except Exception:
        pass
    return False


def _launch_vllm_server(
    name: str,
    model_dir: str,
    host: str,
    port: int,
    served_model_name: Optional[str],
    gpu_util: float,
    max_model_len: Optional[int],
    max_num_seqs: Optional[int],
    extra_flags: Optional[list[str]] = None,
) -> tuple[subprocess.Popen, str]:
    """
    Lanza un servidor vLLM OpenAI-compatible para un modelo dado.
    """
    _free_port_if_needed(host, port)
    log_path = f"vllm_{name}.log"
    cmd = [
        sys.executable,
        "-m",
        "vllm.entrypoints.openai.api_server",
        "--model",
        model_dir,
        "--host",
        host,
        "--port",
        str(port),
        "--gpu-memory-utilization",
        str(gpu_util),
    ]
    if max_model_len is not None:
        cmd += ["--max-model-len", str(max_model_len)]
    if max_num_seqs is not None:
        cmd += ["--max-num-seqs", str(max_num_seqs)]
    if served_model_name:
        cmd += ["--served-model-name", served_model_name]
    if extra_flags:
        cmd += list(extra_flags)

    print(f"\n🚀 Lanzando servidor vLLM [{name}] en puerto {port}")
    print(" ".join(cmd))
    proc = subprocess.Popen(
        cmd,
        stdout=open(log_path, "w"),
        stderr=subprocess.STDOUT,
    )
    return proc, log_path


def start_qwen_vllm_servers(
    model_paths: QwenModelPaths,
    config: Optional[VllmConfig] = None,
    set_env: bool = True,
) -> tuple[VllmEndpoints, VllmServers]:
    """
    Lanza servidores vLLM:

      - LLM (generate, con tool-calling + reasoning de Qwen3)
      - (Opcional) Embeddings (embed)
      - (Opcional) Reranker (score)
    """
    cfg = config or VllmConfig()

    # LLM – flags alineados a Qwen3 + langchain-qwq
    llm_extra_flags: list[str] = [
        "--enable-auto-tool-choice",
        "--tool-call-parser",
        cfg.tool_call_parser,
    ]
    if cfg.enable_reasoning and cfg.reasoning_parser:
        llm_extra_flags += ["--reasoning-parser", cfg.reasoning_parser]

    llm_proc, llm_log = _launch_vllm_server(
        name="language",
        model_dir=model_paths.llm_dir,
        host=cfg.host,
        port=cfg.llm_port,
        served_model_name=cfg.llm_served_name,
        gpu_util=cfg.llm_gpu_util,
        max_model_len=cfg.llm_max_len,
        max_num_seqs=cfg.llm_max_num_seqs,
        extra_flags=llm_extra_flags,
    )

    emb_proc = None
    emb_log = None
    if cfg.start_emb_server:
        emb_proc, emb_log = _launch_vllm_server(
            name="embedding",
            model_dir=model_paths.emb_dir,
            host=cfg.host,
            port=cfg.emb_port,
            served_model_name=cfg.emb_served_name,
            gpu_util=cfg.emb_gpu_util,
            max_model_len=cfg.emb_max_len,
            max_num_seqs=cfg.emb_max_num_seqs,
        )

    rerank_proc = None
    rerank_log = None
    if cfg.start_rerank_server:
        rerank_proc, rerank_log = _launch_vllm_server(
            name="reranker",
            model_dir=model_paths.rerank_dir,
            host=cfg.host,
            port=cfg.rerank_port,
            served_model_name=cfg.rerank_served_name,
            gpu_util=cfg.rerank_gpu_util,
            max_model_len=cfg.rerank_max_len,
            max_num_seqs=cfg.rerank_max_num_seqs,
        )

    llm_base = f"http://{cfg.host}:{cfg.llm_port}/v1"
    emb_base = f"http://{cfg.host}:{cfg.emb_port}/v1" if cfg.start_emb_server else None
    rerank_base = f"http://{cfg.host}:{cfg.rerank_port}/v1" if cfg.start_rerank_server else None

    print("\n⏳ Esperando LLM server...")
    ok_llm = _wait_until_ready(f"{llm_base}/models", llm_proc, llm_log)

    ok_emb = True
    if cfg.start_emb_server and emb_proc is not None and emb_log is not None:
        print("⏳ Esperando Embedding server...")
        ok_emb = _wait_until_ready(f"{emb_base}/models", emb_proc, emb_log)

    ok_rerank = True
    if cfg.start_rerank_server and rerank_proc is not None and rerank_log is not None:
        print("⏳ Esperando Reranker server...")
        ok_rerank = _wait_until_ready(f"{rerank_base}/models", rerank_proc, rerank_log)

    if not ok_llm or not ok_emb or not ok_rerank:
        raise SystemExit("❌ Algún servidor vLLM no quedó listo. Revisa logs.")

    print("\n✅ Servidores vLLM listos.")
    print("\n📋 Modelos en LLM server:")
    print(_url_open_no_proxy(f"{llm_base}/models"))
    if emb_base is not None:
        print("\n📋 Modelos en Embedding server:")
        print(_url_open_no_proxy(f"{emb_base}/models"))
    if rerank_base is not None:
        print("\n📋 Modelos en Reranker server:")
        print(_url_open_no_proxy(f"{rerank_base}/models"))

    if set_env:
        os.environ["VLLM_LLM_API_BASE"] = llm_base
        os.environ["VLLM_API_BASE"] = llm_base
        if emb_base is not None:
            os.environ["VLLM_EMB_API_BASE"] = emb_base
        if rerank_base is not None:
            os.environ["VLLM_RERANK_API_BASE"] = rerank_base

        # Clave dummy: vLLM ignora el valor, sólo requiere que exista.
        os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY", "EMPTY")
        os.environ["LLM_SERVED_NAME"] = cfg.llm_served_name
        os.environ["EMB_SERVED_NAME"] = cfg.emb_served_name
        os.environ["RERANK_SERVED_NAME"] = cfg.rerank_served_name

        print("\n🌐 Bases URL registradas en ENV:")
        print("VLLM_API_BASE       =", os.environ["VLLM_API_BASE"])
        print("VLLM_LLM_API_BASE   =", os.environ["VLLM_LLM_API_BASE"])
        if emb_base is not None:
            print("VLLM_EMB_API_BASE   =", os.environ["VLLM_EMB_API_BASE"])
        if rerank_base is not None:
            print("VLLM_RERANK_API_BASE=", os.environ["VLLM_RERANK_API_BASE"])

    endpoints = VllmEndpoints(
        llm_base_url=llm_base,
        emb_base_url=emb_base,
        rerank_base_url=rerank_base,
    )
    servers = VllmServers(
        llm_proc=llm_proc,
        emb_proc=emb_proc,
        rerank_proc=rerank_proc,
        llm_log_path=llm_log,
        emb_log_path=emb_log,
        rerank_log_path=rerank_log,
    )
    return endpoints, servers


# ─────────────────────────────────────────────
# PlannerConfig + planner LLM (ChatQwenVllm)
# ─────────────────────────────────────────────

STRICT_SYSTEM_TEXT = (
    "Eres el PLANNER de herramientas de un agente de IA.\n"
    "Tu trabajo es LEER con cuidado la petición completa del usuario, "
    "descomponerla en subtareas cuando sea necesario y planificar una o "
    "VARIAS llamadas a herramientas para resolver TODAS las partes.\n\n"
    "Dispones de un conjunto de herramientas (tools). Para cada tool "
    "conoces su nombre, su descripción y sus parámetros.\n\n"
    "INSTRUCCIONES (agnósticas de dominio):\n"
    "1) Usa SIEMPRE herramientas cuando exista alguna relevante.\n"
    "2) Devuelves únicamente tool_calls (llamadas a herramientas), "
    "   nunca una respuesta final en lenguaje natural.\n"
    "3) Si la instrucción tiene varias acciones (por ejemplo: "
    "   'primero A, luego B, al final C'), planifica todas las tools "
    "   necesarias respetando ese orden.\n"
    "4) Si la entrada menciona varios ítems (varios textos, documentos, "
    "   números, entidades, etc.), considera planear llamadas que cubran "
    "   CADA ítem relevante.\n"
    "5) Siempre que haya al menos una herramienta relevante, llama a UNA "
    "   O MÁS herramientas; no respondas sólo con razonamiento interno.\n"
    "6) Sé explícito y coherente en los argumentos de cada tool_call; "
    "   respeta nombres y tipos de parámetros.\n"
    "7) Si una acción requiere varios pasos, divide el trabajo "
    "   en varias tool_calls encadenadas.\n"
    "8) Decide qué herramientas usar únicamente por su descripción.\n"
)

FREE_SYSTEM_TEXT = (
    "Eres el asistente/PLANNER de un agente.\n"
    "Regla principal:\n"
    "- Si el usuario pide conversación, creatividad (poesía, historias), identidad del asistente, "
    "  explicación general o razonamiento que NO requiere datos externos, responde DIRECTO en lenguaje natural.\n"
    "- Usa herramientas SOLO cuando sean necesarias (DB, archivos, web, embeddings, cálculos deterministas, etc.).\n\n"
    "Cuando uses herramientas:\n"
    "- Genera tool_calls correctas (nombre + argumentos) para resolver lo pedido.\n\n"
    "Cuando NO uses herramientas:\n"
    "- NO inventes que necesitas tools.\n"
    "- Devuelve una respuesta final clara y útil.\n"
)


@dataclass
class PlannerConfig:
    """Configuración de alto nivel del planner de herramientas."""
    model_name: str = os.getenv("LLM_SERVED_NAME", "qwen3-0.6b")
    tool_choice: str = os.getenv("PLANNER_TOOL_CHOICE", "required")
    max_retries: int = int(os.getenv("PLANNER_MAX_RETRIES", "1"))
    max_steps: int = int(os.getenv("PLANNER_MAX_STEPS", "16"))
    temperature: float = float(os.getenv("PLANNER_TEMPERATURE", "0.0"))
    policy_mode: str = os.getenv("PLANNER_POLICY_MODE", "tools_strict")

    # Qwen3-style reasoning
    enable_thinking: bool = _str_to_bool(os.getenv("PLANNER_ENABLE_THINKING", "true"))

    # 4) Ajuste “policy prompt”: 2 modos a nivel prompt
    policy_mode: Literal["tools_strict", "free_policies"] = os.getenv(
        "PLANNER_POLICY_MODE",
        "tools_strict",
    )  # "tools_strict" | "free_policies"

    # Prompt base del planner (agnóstico de dominio)
    system_text: str = (
        "Eres el PLANNER de un agente de IA con acceso opcional a herramientas (tools).\n"
        "Tu trabajo es LEER con cuidado la petición completa del usuario, "
        "descomponerla en subtareas cuando sea necesario y decidir si debes "
        "hacer UNA o VARIAS llamadas a herramientas.\n\n"
        "Dispones de un conjunto de herramientas (tools). Para cada tool "
        "conoces su nombre, su descripción y sus parámetros.\n\n"
        "INSTRUCCIONES (agnósticas de dominio):\n"
        "1) Si una tool es necesaria para operar sobre datos externos (DB, archivos, web, "
        "embeddings, búsqueda, parsing, etc.), úsala.\n"
        "2) Si la instrucción tiene varias acciones (por ejemplo: "
        "   'primero A, luego B, al final C'), planifica todas las tools "
        "   necesarias respetando ese orden.\n"
        "3) Si la entrada menciona varios ítems (varios textos, documentos, "
        "   números, entidades, etc.), considera cubrir CADA ítem relevante.\n"
        "4) Sé explícito y coherente en los argumentos de cada tool_call; "
        "   respeta nombres y tipos de parámetros.\n"
        "5) Si una acción requiere varios pasos (por ejemplo: transformar un "
        "   dato y luego consultarlo en otra herramienta), divide el trabajo "
        "   en varias tool_calls encadenadas.\n"
        "6) No supongas un dominio específico (texto, matemáticas, APIs, "
        "   bases de datos…); decide qué herramientas usar únicamente por su "
        "   descripción.\n"
    )


def build_planner_system_message(config: Optional[PlannerConfig] = None) -> SystemMessage:
    cfg = config or PlannerConfig()
    if cfg.policy_mode == "free_policies":
        return SystemMessage(content=FREE_SYSTEM_TEXT)
    return SystemMessage(content=STRICT_SYSTEM_TEXT)

def build_planner_llm(config: Optional[PlannerConfig] = None) -> ChatQwenVllm:
    """
    Construye el LLM planner apuntando al endpoint vLLM (VLLM_API_BASE).
    No asume tools; se configuran fuera con .bind_tools().
    """
    cfg = config or PlannerConfig()

    # Compatibilidad: si solo se definió VLLM_LLM_API_BASE, lo usamos como VLLM_API_BASE.
    if "VLLM_LLM_API_BASE" in os.environ and "VLLM_API_BASE" not in os.environ:
        os.environ["VLLM_API_BASE"] = os.environ["VLLM_LLM_API_BASE"]

    return ChatQwenVllm(
        model=cfg.model_name,
        temperature=cfg.temperature,
        enable_thinking=cfg.enable_thinking,
    )

In [ ]:
#@title agnostic_agent/tools.py
%%writefile agnostic_agent/tools.py
from __future__ import annotations

"""
Catálogo de herramientas (tools) para el Agnostic Deep Agent 2026.

Incluye:
- Tools "toy" (to_upper, word_count, is_palindrome).
- Tools matemáticas seguras (eval_math_expression, sum_numbers, average_numbers).
- Tools de modelos Qwen3 locales:
    - embed_texts              → Qwen3-Embedding (núcleo reutilizable).
    - semantic_search          → búsqueda semántica sobre una lista de textos.
    - semantic_search_in_csv   → búsqueda semántica sobre filas de un CSV.
    - embed_context_tables     → precálculo de embeddings para tablas de contexto.
    - judge_row_with_context   → juicio simple de una fila usando contexto tabular.
    - rerank_qwen3             → Qwen3-Reranker

El registro global TOOL_REGISTRY permite seleccionar tools por nombre
y construir subconjuntos según la configuración (p.ej. setup.yaml).
"""

from typing import List, Dict, Any

from langchain_core.tools import tool

import os
import ast
import operator as _op
import numbers

import torch
from transformers import (
    AutoTokenizer,
    AutoModel,              # Qwen3-Embedding
    AutoModelForCausalLM,   # Qwen3-Reranker (via logits yes/no)
)

import numpy as np
import pandas as pd


# ─────────────────────────────────────────────
# TOOLS "toy"
# ─────────────────────────────────────────────

@tool
def to_upper(text: str) -> str:
    """Convierte el texto a MAYÚSCULAS."""
    return text.upper()


@tool
def word_count(text: str) -> int:
    """Devuelve el número de palabras en el texto."""
    return len([w for w in text.split() if w])


@tool
def is_palindrome(text: str) -> bool:
    """True si el texto (sin espacios/casos) es palíndromo."""
    s = "".join(ch.lower() for ch in text if ch.isalnum())
    return s == s[::-1]


# ─────────────────────────────────────────────
# TOOLS matemáticas (evaluadas en Python)
# ─────────────────────────────────────────────

# Operadores permitidos en la expresión matemática
_ALLOWED_OPS = {
    ast.Add: _op.add,
    ast.Sub: _op.sub,
    ast.Mult: _op.mul,
    ast.Div: _op.truediv,
    ast.Pow: _op.pow,
    ast.Mod: _op.mod,
}


def _eval_ast(node: ast.AST) -> float:
    """Evalúa de forma segura un AST restringido a operaciones aritméticas básicas."""
    if isinstance(node, ast.Num):  # Python <3.8
        return node.n
    if isinstance(node, ast.Constant):  # números en 3.8+
        if isinstance(node.value, (int, float)):
            return node.value
        raise ValueError("Sólo se permiten números en las constantes.")

    if isinstance(node, ast.BinOp):
        op_type = type(node.op)
        if op_type not in _ALLOWED_OPS:
            raise ValueError(f"Operador no permitido: {op_type.__name__}")
        left = _eval_ast(node.left)
        right = _eval_ast(node.right)
        return _ALLOWED_OPS[op_type](left, right)

    if isinstance(node, ast.UnaryOp) and isinstance(node.op, ast.USub):
        return -_eval_ast(node.operand)

    raise ValueError(f"Nodo de AST no permitido: {type(node).__name__}")


@tool
def eval_math_expression(expression: str) -> float:
    """
    Evalúa una expresión matemática sencilla usando Python de forma segura.

    Soporta:
      - suma, resta, multiplicación, división, módulo, potencias
      - paréntesis
      - signos unarios (p.ej. -3)

    Ejemplos válidos:
      "1 + 2 * 3"
      "(10 - 4) / 2"
      "2**3 + 5"

    NOTA:
      El operador de potencia soportado es **, NO ^ (que en Python es XOR).
    """
    try:
        parsed = ast.parse(expression, mode="eval")
        result = _eval_ast(parsed.body)
        return float(result)
    except Exception as exc:
        raise ValueError(
            f"No se pudo evaluar la expresión: {expression!r}. Error: {exc}"
        ) from exc


# ─────────────────────────────────────────────
# Helpers numéricos robustos para sum/avg
# ─────────────────────────────────────────────

def _coerce_to_float(x: Any) -> float:
    """
    Intenta convertir un valor genérico a float.

    Soporta:
    - ints/floats/np.number
    - strings numéricos ("3.14")
    - dicts con claves típicas: "value", "val", "number", "num"
    """
    if isinstance(x, numbers.Number):
        return float(x)

    if isinstance(x, str):
        # Permite strings como "3.14", "42"
        return float(x.strip())

    if isinstance(x, dict):
        for key in ("value", "val", "number", "num"):
            if key in x:
                return _coerce_to_float(x[key])

    raise ValueError(f"No se pudo interpretar {x!r} como número.")


@tool
def sum_numbers(numbers: List[Any]) -> float:
    """
    Devuelve la suma de una lista de números.

    La tool es robusta: acepta tanto números puros como estructuras
    que contengan el número, por ejemplo:

      - [1, 2.5, "3"]
      - [{"value": 10}, {"number": "20"}]

    Para evitar errores de validación Pydantic, el tipo es List[Any]
    y se hace coerción interna a float.
    """
    if not isinstance(numbers, list):
        raise ValueError("El parámetro 'numbers' debe ser una lista.")

    vals = [_coerce_to_float(n) for n in numbers]
    return float(sum(vals))


@tool
def average_numbers(numbers: List[Any]) -> float:
    """
    Devuelve la media aritmética de una lista de números.

    Mismo comportamiento robusto que sum_numbers:
      - [1, 2.5, "3"]
      - [{"value": 10}, {"number": "20"}]
    """
    if not isinstance(numbers, list):
        raise ValueError("El parámetro 'numbers' debe ser una lista.")
    if not numbers:
        raise ValueError("La lista de números está vacía.")

    vals = [_coerce_to_float(n) for n in numbers]
    return float(sum(vals) / len(vals))


# ─────────────────────────────────────────────
# QWEN3-EMBEDDING – Transformers local
# ─────────────────────────────────────────────

_EMB_STATE: Dict[str, Any] = {}


def _ensure_embedding_loaded() -> None:
    """Carga Qwen3-Embedding una sola vez en memoria."""
    global _EMB_STATE
    if _EMB_STATE:
        return

    model_id = os.getenv("EMB_MODEL_ID", "Qwen/Qwen3-Embedding-0.6B")

    # Permite forzar device vía env si quieres:
    #   QWEN_EMB_DEVICE = "cuda" | "cpu"
    forced_device = os.getenv("QWEN_EMB_DEVICE")
    if forced_device in ("cuda", "cpu"):
        device = forced_device
    else:
        use_cuda = (
            os.getenv("QWEN_EMB_USE_CUDA", "0").lower() in ("1", "true", "yes")
            and torch.cuda.is_available()
        )
        device = "cuda" if use_cuda else "cpu"

    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModel.from_pretrained(model_id, trust_remote_code=True)
    model.to(device)
    model.eval()

    max_length = int(os.getenv("QWEN_EMB_MAX_LEN", "512"))

    _EMB_STATE.update(
        {
            "model_id": model_id,
            "device": device,
            "tokenizer": tokenizer,
            "model": model,
            "max_length": max_length,
        }
    )


def _embed_texts_core(inputs: List[str]) -> np.ndarray:
    """
    Núcleo de embeddings: recibe una lista de textos y devuelve un array (n, d).

    Se usa internamente por:
      - embed_texts (tool)
      - semantic_search
      - semantic_search_in_csv
      - embed_context_tables
    """
    _ensure_embedding_loaded()
    state = _EMB_STATE

    tokenizer = state["tokenizer"]
    model = state["model"]
    device = state["device"]
    max_length = state["max_length"]

    if not inputs:
        return np.zeros((0, 0), dtype="float32")

    enc = tokenizer(
        inputs,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt",
    )
    enc = {k: v.to(device) for k, v in enc.items()}

    with torch.no_grad():
        outputs = model(**enc)
        last_hidden = outputs.last_hidden_state  # (batch, seq_len, hidden)
        emb = last_hidden.mean(dim=1)           # (batch, hidden)

    return emb.cpu().numpy()


@tool
def embed_texts(texts: List[str]) -> List[List[float]]:
    """
    Devuelve embeddings Qwen3-Embedding para cada texto, usando Transformers local.

    - Usa EMB_MODEL_ID (por defecto Qwen/Qwen3-Embedding-0.6B).
    - Por defecto corre en CPU (QWEN_EMB_USE_CUDA=0).
    - Devuelve una lista de vectores (list[list[float]]).

    NOTA:
    - Internamente usa _embed_texts_core para compartir el mismo estado
      con otras tools (semantic_search, semantic_search_in_csv, embed_context_tables).
    """
    if isinstance(texts, str):
        inputs = [texts]
    else:
        inputs = list(texts)

    if not inputs:
        return []

    emb = _embed_texts_core(inputs)
    return emb.tolist()


# ─────────────────────────────────────────────
# BÚSQUEDA SEMÁNTICA GENÉRICA (en memoria)
# ─────────────────────────────────────────────

def _cosine_sim_matrix(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    """
    Matriz de similitud coseno entre:
      - a: (n, d)
      - b: (m, d)

    Devuelve: (n, m).
    """
    if a.size == 0 or b.size == 0:
        return np.zeros((a.shape[0], b.shape[0]), dtype="float32")

    a_norm = a / (np.linalg.norm(a, axis=1, keepdims=True) + 1e-8)
    b_norm = b / (np.linalg.norm(b, axis=1, keepdims=True) + 1e-8)
    return np.matmul(a_norm, b_norm.T)


@tool
def semantic_search(
    query: str,
    documents: List[str],
    top_k: int = 5,
) -> List[Dict[str, Any]]:
    """
    Búsqueda semántica simple usando Qwen3-Embedding sobre una lista de textos.

    Pensado para casos como:
      - Dado un query, rankear párrafos, definiciones, cláusulas, etc.
      - Integrar tablas ya "serializadas" a textos (fila → string).

    Parámetros:
      - query: texto de búsqueda.
      - documents: lista de textos candidatos.
      - top_k: número máximo de resultados a devolver.

    Devuelve una lista de dicts:
      [{ "index": int, "document": str, "score": float }, ...]
    ordenada por score descendente.
    """
    if isinstance(documents, str):
        docs = [documents]
    else:
        docs = list(documents)

    if not docs:
        return []

    # Embeddings: query (1, d) y docs (n, d)
    query_emb = _embed_texts_core([query])          # (1, d)
    docs_emb = _embed_texts_core(docs)              # (n, d)

    sims = _cosine_sim_matrix(query_emb, docs_emb)  # (1, n)
    scores = sims[0]

    # Top-k
    top_k = max(1, min(top_k, len(docs)))
    indices = np.argsort(-scores)[:top_k]

    results: List[Dict[str, Any]] = []
    for idx in indices:
        results.append(
            {
                "index": int(idx),
                "document": docs[idx],
                "score": float(scores[idx]),
            }
        )

    return results


# ─────────────────────────────────────────────
# BÚSQUEDA SEMÁNTICA EN CSV (parametrías, diccionarios, etc.)
# ─────────────────────────────────────────────

# Cache por (csv_path, columnas_join) → { "df": DataFrame, "emb": np.ndarray, "texts": List[str] }
_CSV_EMB_CACHE: Dict[str, Any] = {}


def _get_csv_embeddings(
    csv_path: str,
    text_columns: List[str],
) -> Dict[str, Any]:
    """
    Carga (o reutiliza) embeddings por fila para un CSV.

    - csv_path: ruta al CSV (p.ej. parametrias.csv, diccionario_abreviaturas.csv).
    - text_columns: columnas a concatenar para generar el texto base.

    Devuelve un dict:
      {
        "df": DataFrame,
        "emb": np.ndarray (n_filas, d),
        "texts": List[str]  # representación textual de cada fila
      }
    """
    key = f"{os.path.abspath(csv_path)}|{'|'.join(text_columns)}"
    if key in _CSV_EMB_CACHE:
        return _CSV_EMB_CACHE[key]

    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"No se encontró el CSV: {csv_path}")

    df = pd.read_csv(csv_path)

    # Normalizamos text_columns a las que existan
    cols = [c for c in text_columns if c in df.columns]
    if not cols:
        raise ValueError(
            f"Ninguna de las columnas {text_columns!r} existe en el CSV {csv_path!r}."
        )

    texts: List[str] = []
    for _, row in df.iterrows():
        parts = []
        for col in cols:
            val = row.get(col, "")
            if pd.isna(val):
                continue
            parts.append(f"{col}: {val}")
        texts.append(" | ".join(parts))

    emb = _embed_texts_core(texts)  # (n_filas, d)

    payload = {
        "df": df,
        "emb": emb,
        "texts": texts,
    }
    _CSV_EMB_CACHE[key] = payload
    return payload


@tool
def semantic_search_in_csv(
    query: str,
    csv_path: str,
    text_columns: List[str],
    top_k: int = 5,
) -> List[Dict[str, Any]]:
    """
    Búsqueda semántica sobre filas de un CSV usando Qwen3-Embedding.

    Casos típicos:
      - csv_path="parametrias.csv", text_columns=["campo", "descripcion", "regla"]
      - csv_path="diccionario_abreviaturas.csv", text_columns=["abreviatura", "definicion"]

    Parámetros:
      - query: texto de búsqueda (ej. "monto máximo de crédito hipotecario").
      - csv_path: ruta al archivo CSV.
      - text_columns: columnas que se concatenan para representar cada fila.
      - top_k: número máximo de filas a devolver.

    Devuelve:
      [
        {
          "row_index": int,
          "score": float,
          "row": {col: valor, ...},
          "text": "col1: ... | col2: ..."
        },
        ...
      ]
    """
    payload = _get_csv_embeddings(csv_path, text_columns)
    df: pd.DataFrame = payload["df"]
    emb: np.ndarray = payload["emb"]

    if df.empty:
        return []

    # Embedding del query
    query_emb = _embed_texts_core([query])  # (1, d)
    sims = _cosine_sim_matrix(query_emb, emb)[0]  # (n_filas,)

    top_k = max(1, min(top_k, len(df)))
    indices = np.argsort(-sims)[:top_k]

    results: List[Dict[str, Any]] = []
    for idx in indices:
        row_data = df.iloc[int(idx)].to_dict()
        results.append(
            {
                "row_index": int(idx),
                "score": float(sims[idx]),
                "row": row_data,
                "text": payload["texts"][int(idx)],
            }
        )

    return results


# ─────────────────────────────────────────────
# CONTEXTO: precálculo de embeddings de tablas
# ─────────────────────────────────────────────

@tool
def embed_context_tables(
    table_paths: List[str],
    text_columns: Dict[str, List[str]] | None = None,
) -> Dict[str, Any]:
    """
    Precálcula embeddings por fila para varias tablas de contexto (CSV):

      - Parametrías (reglas, umbrales, categorías, etc.).
      - Diccionarios de abreviaturas/definiciones.
      - Otras tablas de contexto que quieras.

    Parámetros:
      - table_paths: lista de rutas a CSVs de contexto.
      - text_columns: mapa opcional {path: [cols,...]} con las columnas
        que se usarán para generar los textos por fila. Si no se indica
        para un path, se usan todas las columnas del CSV.

    Efectos:
      - Llama internamente a _get_csv_embeddings(...) para cada tabla,
        poblando la caché interna _CSV_EMB_CACHE.
      - Reutiliza el mismo modelo de embeddings que embed_texts.

    Devuelve un resumen:

      {
        "tables": [
          {
            "path": "<ruta_csv>",
            "n_rows": int,
            "n_cols": int,
            "text_columns": [ ... ]
          },
          ...
        ],
        "embedding_dim": int | null
      }
    """
    if isinstance(table_paths, str):
        paths = [table_paths]
    else:
        paths = list(table_paths)

    tables_info: List[Dict[str, Any]] = []
    emb_dim = None

    for p in paths:
        # Determinar columnas de texto para este path
        cols = None
        if text_columns and isinstance(text_columns, dict):
            cols = text_columns.get(p)

        if not cols:
            # Si no se especifican columnas, usamos todas
            if not os.path.exists(p):
                raise FileNotFoundError(f"No se encontró el CSV: {p}")
            df_head = pd.read_csv(p, nrows=1)
            cols = list(df_head.columns)

        payload = _get_csv_embeddings(p, cols)
        df = payload["df"]
        emb = payload["emb"]

        if emb.size > 0:
            d = emb.shape[1]
            if emb_dim is None:
                emb_dim = d
            elif emb_dim != d:
                # Si hay discrepancia, mantenemos el primero y no reventamos
                pass

        tables_info.append(
            {
                "path": p,
                "n_rows": int(len(df)),
                "n_cols": int(len(df.columns)),
                "text_columns": cols,
            }
        )

    return {
        "tables": tables_info,
        "embedding_dim": emb_dim,
    }


# ─────────────────────────────────────────────
# QWEN3-RERANKER – Transformers local
# ─────────────────────────────────────────────

_RERANK_STATE: Dict[str, Any] = {}


def _ensure_reranker_loaded() -> None:
    """Carga Qwen3-Reranker una sola vez en memoria."""
    global _RERANK_STATE
    if _RERANK_STATE:
        return

    model_id = os.getenv("RERANK_MODEL_ID", "Qwen/Qwen3-Reranker-0.6B")

    forced_device = os.getenv("QWEN_RERANK_DEVICE")
    if forced_device in ("cuda", "cpu"):
        device = forced_device
    else:
        use_cuda = (
            os.getenv("QWEN_RERANK_USE_CUDA", "0").lower() in ("1", "true", "yes")
            and torch.cuda.is_available()
        )
        device = "cuda" if use_cuda else "cpu"

    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.padding_side = "left"
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # Importante: trust_remote_code=True para Qwen3-Reranker
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        trust_remote_code=True,
    )
    model.to(device)
    model.eval()

    true_token_id = tokenizer("yes", add_special_tokens=False).input_ids[0]
    false_token_id = tokenizer("no", add_special_tokens=False).input_ids[0]

    max_length = int(os.getenv("QWEN_RERANK_MAX_LEN", "1024"))

    _RERANK_STATE.update(
        {
            "model_id": model_id,
            "device": device,
            "tokenizer": tokenizer,
            "model": model,
            "true_token_id": true_token_id,
            "false_token_id": false_token_id,
            "max_length": max_length,
        }
    )


def _format_rerank_prompts(
    query: str,
    docs: List[str],
    instruction: str,
) -> List[str]:
    prompts: List[str] = []
    for doc in docs:
        text = (
            "You are a relevance judge. "
            "Decide if the document answers the query.\n\n"
            f"Instruction: {instruction}\n"
            f"Query: {query}\n"
            f"Document: {doc}\n\n"
            "Answer with 'yes' if it is relevant, otherwise 'no'."
        )
        prompts.append(text)
    return prompts


@tool
def rerank_qwen3(query: str, documents: List[str]) -> List[Dict[str, Any]]:
    """
    Usa Qwen3-Reranker (local, vía Transformers) para ordenar documentos por relevancia.

    - Usa RERANK_MODEL_ID (por defecto Qwen/Qwen3-Reranker-0.6B).
    - Por defecto corre en CPU (QWEN_RERANK_USE_CUDA=0).
    - Devuelve [{index, document, score}] ordenado por score desc.
    """
    _ensure_reranker_loaded()
    state = _RERANK_STATE

    tokenizer = state["tokenizer"]
    model = state["model"]
    device = state["device"]
    true_token_id = state["true_token_id"]
    false_token_id = state["false_token_id"]
    max_length = state["max_length"]

    if isinstance(documents, str):
        docs = [documents]
    else:
        docs = list(documents)

    if not docs:
        return []

    instruction = os.getenv(
        "QWEN_RERANK_INSTRUCT",
        "Given a web search query, rank documents by how well they answer the query.",
    )

    prompts = _format_rerank_prompts(query, docs, instruction)

    enc = tokenizer(
        prompts,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt",
    )
    enc = {k: v.to(device) for k, v in enc.items()}

    with torch.no_grad():
        outputs = model(**enc)
        logits = outputs.logits[:, -1, :]   # (batch, vocab)
        yes_logits = logits[:, true_token_id]
        no_logits = logits[:, false_token_id]
        stacked = torch.stack([no_logits, yes_logits], dim=-1)  # (batch, 2)
        probs = torch.nn.functional.softmax(stacked, dim=-1)[:, 1].tolist()

    results: List[Dict[str, Any]] = []
    for idx, (doc, score) in enumerate(zip(docs, probs)):
        results.append(
            {
                "index": idx,
                "document": doc,
                "score": float(score),
            }
        )

    results.sort(key=lambda x: x["score"], reverse=True)
    return results


# ─────────────────────────────────────────────
# JUICIO FILA + CONTEXTO (parametrías / diccionarios)
# ─────────────────────────────────────────────

@tool
def judge_row_with_context(
    row: Dict[str, Any],
    param_hits: List[Dict[str, Any]] | None = None,
    glossary_hits: List[Dict[str, Any]] | None = None,
) -> Dict[str, Any]:
    """
    Aplica un juicio simple sobre una fila de atributos usando hits de contexto.

    Pensado para el caso:
      - row: fila de atributos de un contrato (input A).
      - param_hits: resultados de semantic_search_in_csv sobre parametrías.
      - glossary_hits: resultados de semantic_search_in_csv sobre diccionario
        de abreviaturas / definiciones.

    NO usa modelos de lenguaje dentro de la tool; es puramente determinista
    y genérico. El LLM puede decidir cómo interpretar el resultado.

    Reglas sencillas:
      - Si NO hay hits ni en param_hits ni en glossary_hits → "review_required".
      - Si hay al menos un hit en alguna de las dos → "ok".
      - Se devuelve también un pequeño resumen de motivos.

    Devuelve un dict como:

      {
        "contract_id": str | int | null,
        "judgement": "ok" | "review_required",
        "reasons": [str, ...],
        "row": row,
        "param_hits": [...],
        "glossary_hits": [...],
      }
    """
    param_hits = param_hits or []
    glossary_hits = glossary_hits or []

    # Heurística para encontrar un identificador de contrato/fila
    contract_id = (
        row.get("numero_contrato")
        or row.get("numero de contrato")
        or row.get("num_contrato")
        or row.get("contract_number")
        or row.get("contract_id")
        or row.get("id")
    )

    has_context = bool(param_hits or glossary_hits)
    judgement = "ok" if has_context else "review_required"

    reasons: List[str] = []
    if not has_context:
        reasons.append(
            "No se encontraron coincidencias ni en las parametrías ni en el diccionario; "
            "se recomienda revisión manual."
        )
    else:
        if param_hits:
            best_param = param_hits[0]
            desc = best_param.get("text") or str(best_param.get("row", ""))[:200]
            reasons.append(
                f"Se encontraron al menos {len(param_hits)} filas relevantes en la tabla de parametrías. "
                f"Ejemplo: {desc}"
            )
        if glossary_hits:
            best_gl = glossary_hits[0]
            desc = best_gl.get("text") or str(best_gl.get("row", ""))[:200]
            reasons.append(
                f"Se encontraron al menos {len(glossary_hits)} filas relevantes en el diccionario. "
                f"Ejemplo: {desc}"
            )

    return {
        "contract_id": contract_id,
        "judgement": judgement,
        "reasons": reasons,
        "row": row,
        "param_hits": param_hits,
        "glossary_hits": glossary_hits,
    }


# ─────────────────────────────────────────────
# Registro de tools
# ─────────────────────────────────────────────

TOOL_REGISTRY: Dict[str, Any] = {
    "to_upper": to_upper,
    "word_count": word_count,
    "is_palindrome": is_palindrome,
    "eval_math_expression": eval_math_expression,
    "sum_numbers": sum_numbers,
    "average_numbers": average_numbers,
    "embed_texts": embed_texts,
    "semantic_search": semantic_search,
    "semantic_search_in_csv": semantic_search_in_csv,
    "embed_context_tables": embed_context_tables,
    "rerank_qwen3": rerank_qwen3,
    "judge_row_with_context": judge_row_with_context,
}


def get_default_tools(enabled_names: List[str] | None = None) -> List[Any]:
    """
    Tools por defecto del agente agnóstico.

    - Si enabled_names es None → devuelve TODAS las tools registradas.
    - Si enabled_names es una lista → sólo devuelve las que estén en TOOL_REGISTRY.
    """
    if enabled_names is None:
        return list(TOOL_REGISTRY.values())
    return [TOOL_REGISTRY[name] for name in enabled_names if name in TOOL_REGISTRY]


def get_tools_by_names(names: List[str]) -> List[Any]:
    """
    Permite seleccionar tools por nombre desde TOOL_REGISTRY.
    Equivalente a get_default_tools(names), se mantiene por claridad semántica.
    """
    return get_default_tools(names)


In [ ]:
#@title agnostic_agent/communication.py
%%writefile agnostic_agent/communication.py
from __future__ import annotations

"""
Contratos de I/O de alto nivel del Agnostic Deep Agent 2026.

Este módulo define:
- AgentInput:  payload de entrada para el agente.
- ToolRun:     traza de ejecución de herramientas (para vistas deep/dev).
- AgentSummary:resumen por fases del pipeline interno.
- AgentView:   una vista del resultado para un "rol" (user / deep / dev).
- AgentOutput: objeto de salida unificado con las tres vistas.

NOTAS:
- Los contratos internos más ricos (AnalyzerIntent, PlannerPlan, etc.)
  vivirán en `schemas.py`. Aquí sólo se define I/O de "frontera".
- El campo canónico es `user_prompt`, pero se acepta también `user_text`
  como alias para compatibilidad.
"""

from typing import Any, Dict, List, Optional

from pydantic import BaseModel, Field, ConfigDict


# ─────────────────────────────────────────────
# INPUT
# ─────────────────────────────────────────────

class AgentInput(BaseModel):
    """
    Input "oficial" del agente.

    - user_prompt: texto natural del usuario (nombre canónico).
    - session_id: id opcional de sesión (por si quieres manejar múltiples).
    - kb_names: lista de KBs a considerar (FAOSTAT_WORLD, etc.).
    - metadata: extras arbitrarios (canal, idioma detectado, flags, etc.).
    - data_payload: payload estructurado para este turno
        (por ejemplo, fila de atributos, texto OCR, tablas, etc.).

    Compatibilidad:
    - También acepta el alias `user_text` al instanciar el modelo.
    """

    # Permite poblar usando el nombre de campo o el alias
    model_config = ConfigDict(populate_by_name=True)

    user_prompt: str = Field(
        ...,
        description="Texto de entrada del usuario.",
        alias="user_text",  # compat: antes usábamos `user_text`
    )
    session_id: Optional[str] = Field(
        default=None,
        description="Identificador opcional de sesión.",
    )
    kb_names: List[str] = Field(
        default_factory=list,
        description="Nombres de Knowledge Bases relevantes.",
    )
    metadata: Dict[str, Any] = Field(
        default_factory=dict,
        description="Metadatos arbitrarios asociados a la petición.",
    )
    data_payload: Dict[str, Any] = Field(
        default_factory=dict,
        description=(
            "Payload estructurado asociado al turno. "
            "Ejemplos: una fila de atributos, texto OCR, "
            "tablas de entrada, IDs de contrato, etc."
        ),
    )


# ─────────────────────────────────────────────
# TRAZAS DE TOOLS + RESUMEN
# ─────────────────────────────────────────────

class ToolRun(BaseModel):
    """
    Ejecución de una herramienta.

    - id: tool_call_id interno o identificador de step.
    - name: nombre de la tool.
    - args: argumentos con los que se llamó (ya resueltos).
    - output: salida cruda (sin postprocesar).
    """
    id: str
    name: str
    args: Dict[str, Any]
    output: Any


class AgentSummary(BaseModel):
    """
    Resumen estructurado del pipeline:

      ANALYZER → PLANNER → EXECUTOR → CATCHER → VALIDATOR → MEMORY → SUMMARIZER
    """
    analyzer: str = ""
    planner: str = ""
    executor: str = ""
    catcher: str = ""
    validator: str = ""   # nuevo nodo explícito
    memory: str = ""      # resumen de escritura/uso de memoria
    summarizer: str = ""
    final_answer: str = ""


# ─────────────────────────────────────────────
# VISTAS POR ROL
# ─────────────────────────────────────────────

class AgentView(BaseModel):
    """
    Vista del agente para un "rol" (dev / deep / user).

    - final_answer: texto final para ese rol.
    - summary: breakdown interno del pipeline.
    - tool_runs: ejecuciones de tools (ya normalizadas).
    - raw_state: estado crudo del grafo (según rol, puede ir vacío).
    """
    final_answer: str = ""
    summary: Optional[AgentSummary] = None
    tool_runs: List[ToolRun] = Field(default_factory=list)
    raw_state: Dict[str, Any] = Field(default_factory=dict)


# ─────────────────────────────────────────────
# OUTPUT UNIFICADO
# ─────────────────────────────────────────────

class AgentOutput(BaseModel):
    """
    Output "oficial" del agente (UN SOLO OBJETO):

      {
        "dev_out":  AgentView(...),  # traza completa (pipeline + raw_state)
        "deep_out": AgentView(...),  # resumen por sección
        "user_out": AgentView(...),  # respuesta 1:1 para usuario final
      }
    """
    dev_out: AgentView
    deep_out: AgentView
    user_out: AgentView

    def to_dict(self) -> Dict[str, Any]:
        """
        Devuelve un dict puro {dev_out, deep_out, user_out}, listo para JSON / logging.
        """
        return self.model_dump()

    # ─────────────────────────────────────
    # Helper para construir desde AgentState
    # ─────────────────────────────────────
    @classmethod
    def from_state(cls, state: Dict[str, Any]) -> "AgentOutput":
        """
        Crea un AgentOutput a partir del estado interno del grafo.

        Convenciones esperadas (pero tolerantes):
        - state["user_out"], state["deep_out"], state["dev_out"]:
            respuestas finales (str) por rol.
          Alternativamente:
            state["user_answer"], state["deep_answer"], state["dev_answer"].
        - state["tool_runs"]:
            lista de ToolRun, dicts o estructuras similares.
        - summaries opcionales:
            puedes mapear campos del state a AgentSummary si lo deseas
            vía `state["pipeline_summary"]`.
        """
        # Tool runs normalizados
        tool_runs_norm: List[ToolRun] = []
        raw_tool_runs = state.get("tool_runs", []) or []

        for tr in raw_tool_runs:
            if isinstance(tr, ToolRun):
                tool_runs_norm.append(tr)
            elif isinstance(tr, BaseModel):
                tool_runs_norm.append(ToolRun.model_validate(tr.model_dump()))
            elif isinstance(tr, dict):
                # Puede venir con claves extra, las ignoramos
                tool_runs_norm.append(ToolRun.model_validate(tr))
            else:
                # Si es algo raro, lo metemos como output genérico
                tool_runs_norm.append(
                    ToolRun(
                        id=str(len(tool_runs_norm)),
                        name="unknown",
                        args={},
                        output=tr,
                    )
                )

        # Summary opcional, si el grafo lo llena
        summary: Optional[AgentSummary] = None
        summary_data = state.get("pipeline_summary")
        if isinstance(summary_data, AgentSummary):
            summary = summary_data
        elif isinstance(summary_data, BaseModel):
            summary = AgentSummary.model_validate(summary_data.model_dump())
        elif isinstance(summary_data, dict):
            summary = AgentSummary.model_validate(summary_data)

        # Vistas
        user_view = AgentView(
            final_answer=state.get("user_out", "") or state.get("user_answer", ""),
            summary=summary,
            tool_runs=[],  # por defecto no cargamos tool_runs en la vista de usuario
            raw_state={},
        )

        deep_view = AgentView(
            final_answer=state.get("deep_out", "") or state.get("deep_answer", ""),
            summary=summary,
            tool_runs=tool_runs_norm,
            raw_state={},  # opcionalmente podrías incluir fragmentos de estado aquí
        )

        dev_view = AgentView(
            final_answer=state.get("dev_out", "") or state.get("dev_answer", ""),
            summary=summary,
            tool_runs=tool_runs_norm,
            raw_state=state,  # vista de desarrollo: estado crudo completo
        )

        return cls(dev_out=dev_view, deep_out=deep_view, user_out=user_view)


In [ ]:
#@title agnostic_agent/context.py
%%writefile agnostic_agent/context.py
from __future__ import annotations

"""
Contexto externo y Knowledge Bases para el Agnostic Deep Agent 2026.

Este módulo define:
- KnowledgeBase: descriptor ligero de una KB (faostat, sql, vector, api, etc.).
- get_default_context(): lista base de KBs disponibles (opcionalmente desde setup.yaml).
- get_kb_by_names(): helper para filtrar KBs según AgentInput.kb_names.
- build_kb_from_paths(): helper para crear KBs a partir de rutas (CSV, SQLite, etc.).
- build_kb_from_setup(): helper para mapear la sección `knowledge_bases`
  de setup.yaml a objetos KnowledgeBase.

Casos típicos:
- kind="sqlite"      → BD tabular (SQLite clásica).
- kind="sqlite-vec"  → VDB vectorial sobre sqlite-vec.
- kind="table"       → tabla externa (CSV, parquet) tratada como KB tabular.
"""

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Iterable
from pathlib import Path


__all__ = [
    "KnowledgeBase",
    "build_kb_from_paths",
    "build_kb_from_setup",
    "get_default_context",
    "get_kb_by_names",
]


@dataclass
class KnowledgeBase:
    """
    Descriptor simple de una KB tabular/vectorial/etc.

    - name: identificador único de la KB (ej. 'FAOSTAT_WORLD').
    - kind: tipo de backend (ej. 'sqlite', 'sqlite-vec', 'table', 'api', ...).
    - config: parámetros para conectar (rutas, DSNs, tablas, índices, etc.).

      Convenciones suaves para `config` (no obligatorias, pero recomendadas):
      - "path": ruta a archivo o DB (CSV, .db, parquet, etc.).
      - "table": nombre de tabla (si aplica, p.ej. en SQLite).
      - "role": rol lógico ('param_table', 'rules_table', 'dictionary', ...).
    """
    name: str
    kind: str = "generic"
    config: Dict[str, Any] = field(default_factory=dict)

    # Helpers de conveniencia (no obligatorios)
    @property
    def path(self) -> Optional[str]:
        """Ruta principal asociada a la KB (si existe en config)."""
        return self.config.get("path")

    @property
    def table(self) -> Optional[str]:
        """Nombre de tabla asociado (si aplica en backends tabulares)."""
        return self.config.get("table")

    @property
    def role(self) -> Optional[str]:
        """Rol lógico de la KB (param_table, rules_table, dictionary, ...)."""
        return self.config.get("role")


# ─────────────────────────────────────────────
# Builders de KB
# ─────────────────────────────────────────────

def build_kb_from_paths(
    paths: Iterable[str],
    *,
    kind: str = "table",
    default_role: Optional[str] = None,
) -> List[KnowledgeBase]:
    """
    Crea KnowledgeBase a partir de una lista de rutas (CSV, parquet, etc.).

    Pensado para casos como:
      context = ["parametrias.csv", "diccionario_abreviaturas.csv"]

    Ejemplo:
        kbs = build_kb_from_paths(
            ["parametrias.csv", "diccionario.csv"],
            kind="table",
        )

    Cada KB tendrá:
      - name: nombre derivado del stem del archivo (MAYÚSCULAS).
      - kind: el proporcionado (por defecto "table").
      - config: {"path": <ruta>, "role": default_role} si se pasa.
    """
    kb_list: List[KnowledgeBase] = []
    for p in paths:
        if not p:
            continue
        path_obj = Path(p)
        name = path_obj.stem.upper()
        cfg: Dict[str, Any] = {"path": str(path_obj)}
        if default_role is not None:
            cfg["role"] = default_role
        kb_list.append(
            KnowledgeBase(
                name=name,
                kind=kind,
                config=cfg,
            )
        )
    return kb_list


def build_kb_from_setup(setup_cfg: Dict[str, Any]) -> List[KnowledgeBase]:
    """
    Construye una lista de KnowledgeBase a partir de la sección `knowledge_bases`
    de setup.yaml ya cargado en un dict.

    Forma esperada (tolerante):

    knowledge_bases:
      - name: PARAM_TABLE
        kind: table
        config:
          path: /content/parametrias.csv
          role: param_table

      - name: RULES_TABLE
        kind: table
        config:
          path: /content/reglas_calidad.csv
          role: rules_table

      - name: FAOSTAT_SQLITE
        kind: sqlite
        config:
          db_path: /content/faostat_world.db

    También acepta formato dict:

    knowledge_bases:
      PARAM_TABLE:
        kind: table
        config:
          path: /content/parametrias.csv
    """
    kb_section = setup_cfg.get("knowledge_bases") or []
    kb_list: List[KnowledgeBase] = []

    # Normalizamos a lista de dicts
    if isinstance(kb_section, dict):
        items: List[Dict[str, Any]] = []
        for name, cfg in kb_section.items():
            if not isinstance(cfg, dict):
                continue
            item = dict(cfg)
            item.setdefault("name", name)
            items.append(item)
    elif isinstance(kb_section, list):
        items = [x for x in kb_section if isinstance(x, dict)]
    else:
        items = []

    for item in items:
        name = item.get("name")
        if not name:
            continue
        kind = item.get("kind", "generic")
        config = item.get("config") or {}
        if not isinstance(config, dict):
            config = {}
        kb_list.append(
            KnowledgeBase(
                name=str(name),
                kind=str(kind),
                config=config,
            )
        )

    return kb_list


# ─────────────────────────────────────────────
# Contexto por defecto
# ─────────────────────────────────────────────

def get_default_context(
    setup_cfg: Optional[Dict[str, Any]] = None,
) -> List[KnowledgeBase]:
    """
    Devuelve la lista de KBs disponibles por defecto.

    Modos de uso:

      # 1) Sin setup.yaml (modo antiguo / hardcoded)
      kbs = get_default_context()

      # 2) Pasando el dict de setup.yaml ya cargado
      from pathlib import Path
      import yaml

      with Path("setup.yaml").open("r", encoding="utf-8") as f:
          cfg = yaml.safe_load(f) or {}

      kbs = get_default_context(cfg)

    Si `setup_cfg` se pasa y contiene `knowledge_bases`,
    se usa `build_kb_from_setup(setup_cfg)`.

    Si no, se devuelve una lista vacía (o podrías hardcodear aquí KBs
    globales tipo FAOSTAT_SQLITE/FAOSTAT_VECTOR).
    """
    if setup_cfg:
        kbs = build_kb_from_setup(setup_cfg)
        if kbs:
            return kbs

    # Ejemplo de hardcode si quisieras algo global:
    # return [
    #     KnowledgeBase(
    #         name="FAOSTAT_SQLITE",
    #         kind="sqlite",
    #         config={"db_path": "/content/faostat_world.db"},
    #     ),
    #     KnowledgeBase(
    #         name="FAOSTAT_VECTOR",
    #         kind="sqlite-vec",
    #         config={
    #             "db_path": "/content/faostat_world.db",
    #             "table": "faostat_vec",
    #         },
    #     ),
    # ]
    return []


# ─────────────────────────────────────────────
# Filtro por nombres
# ─────────────────────────────────────────────

def get_kb_by_names(
    kb_names: List[str],
    all_kb: Optional[List[KnowledgeBase]] = None,
) -> List[KnowledgeBase]:
    """
    Devuelve las KnowledgeBase cuyo name esté en kb_names.

    - kb_names: nombres solicitados (p.ej. AgentInput.kb_names).
    - all_kb:  lista total de KBs disponibles; si es None, usamos get_default_context().

    Si kb_names está vacío, se devuelve all_kb completa.
    """
    kb_list = all_kb if all_kb is not None else get_default_context()
    if not kb_names:
        return kb_list

    name_set = set(kb_names)
    return [kb for kb in kb_list if kb.name in name_set]


In [ ]:
#@title agnostic_agent/logic.py
%%writefile agnostic_agent/logic.py
from __future__ import annotations

"""
Lógica principal (grafo LangGraph) del Agnostic Deep Agent.

Sub-grafos actuales:
- ANALYZER  → descompone el prompt (rule-based sencillo por ahora).
- PLANNER   → usa Planner LLM (Qwen3+vLLM) para generar tool_calls.
- EXECUTOR  → ejecuta tools reales (LangChain tools).
- CATCHER   → normaliza las salidas de tools a una lista de runs.
- SUMMARIZER→ construye:
    - respuesta final en modo usuario (user_answer),
    - resumen técnico del pipeline (para vistas deep/dev).
- VALIDATOR → revisa si la respuesta parece cubrir todo lo pedido.

Notas:
- Este módulo sigue usando TypedDict; todavía no está cableado
  a los modelos Pydantic de `schemas.py`.
- Ya integra memoria y kb_names en el planner, y deja
  dev_out / deep_out / user_out en el estado.
- Está pensado para casos donde el agente cruza:
    * una tabla de atributos (input A, p.ej. filas de contratos),
    * con tablas de contexto (input B, p.ej. parametrías y
      diccionarios de abreviaturas/definiciones),
    * y, opcionalmente, documentos (OCR de contratos) vía tools
      como semantic_search_in_csv + rerank_qwen3.
"""

from dataclasses import dataclass
from typing import Any, Dict, List, Optional, Callable, Tuple
import json
import re
import uuid
import xml.etree.ElementTree as ET

from typing_extensions import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    ToolMessage,
    AnyMessage,
    SystemMessage,
)

from .capabilities import PlannerConfig, build_planner_system_message


# ─────────────────────────────────────────────
# Tipos de alto nivel para el "program state"
# ─────────────────────────────────────────────

class AnalyzerResult(TypedDict, total=False):
    input_payload: Dict[str, Any]
    propositional_logic: str
    subqueries: List[str]
    subqueries_logic: List[str]


class PlannerTrajectory(TypedDict, total=False):
    subquery: str
    description: str


class ExecutorStep(TypedDict, total=False):
    tool_call_id: str
    tool_name: str
    args: Dict[str, Any]


class SummaryDict(TypedDict, total=False):
    analyzer: str
    planner: str
    executor: str
    catcher: str
    summarizer: str
    final_answer: str


class ValidatorResult(TypedDict, total=False):
    all_covered: bool
    reasoning: str


class State(TypedDict, total=False):
    """
    Estado del grafo (versión 0.2):

    - messages: historial de LangChain Messages.
    - analyzer: resultado ligero del ANALYZER rule-based.
    - planner_trajs: trazas de planificación del PLANNER.
    - executor_steps: pasos efectivamente ejecutados (EXECUTOR).
    - tool_runs: lista de runs normalizados (CATCHER).
    - summary / pipeline_summary: SummaryDict de todo el pipeline.
    - validator: ValidatorResult simple (cobertura / razonamiento).
    - user_prompt / session_id / kb_names / memory_context:
        metadatos que llegan desde Agent (o el llamador).
    - dev_out / deep_out / user_out:
        vistas finales que el Agent puede usar directamente.
    - llm_raw_out / llm_clean_out:
        invariantes para salida directa del modelo (sin tools),
        donde llm_clean_out = llm_raw_out sin <think>...</think>.
    """
    messages: Annotated[List[AnyMessage], add_messages]
    analyzer: Optional[AnalyzerResult]
    planner_trajs: List[PlannerTrajectory]
    executor_steps: List[ExecutorStep]
    tool_runs: List[Dict[str, Any]]
    summary: Optional[SummaryDict]
    pipeline_summary: Optional[SummaryDict]
    validator: Optional[ValidatorResult]

    # Metadatos / contexto
    user_prompt: Optional[str]
    session_id: Optional[str]
    kb_names: List[str]
    memory_context: Optional[Dict[str, Any]]

    # Vistas finales (pueden ser rellenadas por SUMMARIZER)
    dev_out: Optional[str]
    deep_out: Optional[str]
    user_out: Optional[str]

    # Invariantes de salida (para modo sin tools)
    llm_raw_out: Optional[str]
    llm_clean_out: Optional[str]


# ─────────────────────────────────────────────
# Planner runtime helpers (tool_calls)
# ─────────────────────────────────────────────

def _coerce_content_str(content: Any) -> str:
    if isinstance(content, str):
        return content
    if isinstance(content, list):
        parts = []
        for p in content:
            if isinstance(p, dict):
                parts.append(p.get("text", "") or p.get("content", "") or "")
            else:
                parts.append(str(p))
        return "".join(parts)
    return "" if content is None else str(content)


def _parse_args_maybe_json(x: Any) -> dict:
    if isinstance(x, dict):
        return x
    if isinstance(x, str):
        try:
            obj = json.loads(x)
            return obj if isinstance(obj, dict) else {}
        except Exception:
            return {}
    return {}


def _normalize_toolcalls_list(raw_calls: Any) -> List[Dict[str, Any]]:
    """
    Normaliza múltiples formatos a:
      [{"id": ..., "name": ..., "args": {...}}, ...]
    """
    norm: List[Dict[str, Any]] = []
    if not raw_calls:
        return norm

    # ✅ robustez: a veces viene dict o un objeto suelto
    if isinstance(raw_calls, dict):
        raw_calls = [raw_calls]
    elif not isinstance(raw_calls, list):
        raw_calls = [raw_calls]

    for c in raw_calls:
        if isinstance(c, dict):
            fn = c.get("function") or {}
            name = c.get("name") or fn.get("name") or c.get("tool_name")
            if "args" in c:
                args_raw = c.get("args")
            else:
                args_raw = fn.get("arguments") or c.get("arguments") or c.get("parameters")
            id_ = c.get("id") or c.get("tool_call_id")
        else:
            fn = getattr(c, "function", None)
            name = (
                getattr(c, "name", None)
                or (getattr(fn, "name", None) if fn else None)
                or getattr(c, "tool_name", None)
            )
            args_raw = (
                getattr(c, "args", None)
                or (getattr(fn, "arguments", None) if fn else None)
                or getattr(c, "arguments", None)
                or getattr(c, "parameters", None)
            )
            id_ = getattr(c, "id", None) or getattr(c, "tool_call_id", None)

        args = _parse_args_maybe_json(args_raw)
        if name:
            norm.append(
                {
                    "id": id_ or f"call_{uuid.uuid4().hex}",
                    "name": name,
                    "args": args,
                }
            )
    return norm


# ─────────────────────────────────────────────
# ✅ XML fallback robusto (Qwen XML)
# ─────────────────────────────────────────────

def _scan_balanced_json(s: str, i: int) -> Tuple[Optional[str], int]:
    """
    Escanea desde s[i] (debe ser '{') y devuelve (json_str, next_index)
    contando llaves y respetando strings/escapes.
    """
    if i < 0 or i >= len(s) or s[i] != "{":
        return None, i

    depth = 0
    in_str = False
    esc = False
    start = i

    while i < len(s):
        c = s[i]
        if in_str:
            if esc:
                esc = False
            elif c == "\\":
                esc = True
            elif c == '"':
                in_str = False
        else:
            if c == '"':
                in_str = True
            elif c == "{":
                depth += 1
            elif c == "}":
                depth -= 1
                if depth == 0:
                    return s[start : i + 1], i + 1
        i += 1

    return None, i


def _extract_tool_calls_via_etree(text: str) -> List[Dict[str, Any]]:
    """
    Extrae <tool_call>...</tool_call> como dicts (JSON dentro) usando XML real.
    """
    wrapped = f"<root>{text}</root>"
    try:
        root = ET.fromstring(wrapped)
    except ET.ParseError:
        return []

    out: List[Dict[str, Any]] = []
    for node in root.findall(".//tool_call"):
        raw = "".join(node.itertext()).strip()
        if not raw:
            continue

        # JSON directo
        try:
            obj = json.loads(raw)
            if isinstance(obj, dict):
                out.append(obj)
            elif isinstance(obj, list):
                out.extend([it for it in obj if isinstance(it, dict)])
            continue
        except Exception:
            pass

        # brace-scan dentro del texto del tag
        j = raw.find("{")
        if j != -1:
            js, _ = _scan_balanced_json(raw, j)
            if js:
                try:
                    obj2 = json.loads(js)
                    if isinstance(obj2, dict):
                        out.append(obj2)
                except Exception:
                    pass

    return out


def _extract_tool_calls_via_xmlish_bracescan(text: str) -> List[Dict[str, Any]]:
    """
    Cuando el XML viene malformado, buscamos bloques <tool_call>...</tool_call>
    y adentro hacemos JSON parse o brace-scan.
    """
    out: List[Dict[str, Any]] = []
    tag_open = "<tool_call>"
    tag_close = "</tool_call>"

    pos = 0
    while True:
        a = text.find(tag_open, pos)
        if a == -1:
            break
        b = text.find(tag_close, a)
        if b == -1:
            break

        chunk = text[a + len(tag_open) : b].strip()

        # JSON directo
        try:
            obj = json.loads(chunk)
            if isinstance(obj, dict):
                out.append(obj)
            elif isinstance(obj, list):
                out.extend([it for it in obj if isinstance(it, dict)])
        except Exception:
            # brace-scan
            j = chunk.find("{")
            if j != -1:
                js, _ = _scan_balanced_json(chunk, j)
                if js:
                    try:
                        obj2 = json.loads(js)
                        if isinstance(obj2, dict):
                            out.append(obj2)
                    except Exception:
                        pass

        pos = b + len(tag_close)

    return out


def _extract_qwen_xml_calls(ai_msg: AIMessage) -> List[Dict[str, Any]]:
    """
    Fallback robusto para Qwen3-XML:
      1) intenta XML real (ElementTree) con wrapper <root>
      2) si falla (XML roto), usa búsqueda xml-ish + brace-scan
    Luego normaliza a {"id","name","args"} (misma forma que el resto).
    """
    text = _coerce_content_str(getattr(ai_msg, "content", ""))
    if "<tool_call" not in text:
        return []

    parsed = _extract_tool_calls_via_etree(text)
    if not parsed:
        parsed = _extract_tool_calls_via_xmlish_bracescan(text)

    calls: List[Dict[str, Any]] = []
    for obj in parsed:
        if not isinstance(obj, dict):
            continue
        name = obj.get("name") or obj.get("tool_name")
        args_raw = obj.get("arguments") or obj.get("args") or obj.get("parameters") or {}
        args = _parse_args_maybe_json(args_raw)
        if name:
            calls.append(
                {
                    "id": f"call_{uuid.uuid4().hex}",
                    "name": name,
                    "args": args,
                }
            )
    return calls


def extract_tool_calls(ai_msg: AIMessage) -> List[Dict[str, Any]]:
    """
    API robusta para obtener tool_calls de un AIMessage.
    Compatible con:
    - tool_calls nativos (OpenAI / qwen)
    - additional_kwargs["tool_calls"]
    - XML qwen3-XML (<tool_call>{...}</tool_call>)
    """
    if not isinstance(ai_msg, AIMessage):
        return []

    tc = getattr(ai_msg, "tool_calls", None)
    norm = _normalize_toolcalls_list(tc)
    if norm:
        return norm

    addkw = getattr(ai_msg, "additional_kwargs", {}) or {}
    tc2 = addkw.get("tool_calls")
    norm2 = _normalize_toolcalls_list(tc2)
    if norm2:
        return norm2

    return _extract_qwen_xml_calls(ai_msg)


def call_planner_with_retry(
    planner_llm,
    system_message: SystemMessage,
    user_or_history_messages: List[AnyMessage],
    planner_config: PlannerConfig,
    extra_system_messages: Optional[List[SystemMessage]] = None,
) -> AIMessage:
    """
    Llama al planner_llm con un SystemMessage fijo + historial (+ contextos extra).
    Si no produce tool_calls, reintenta hasta max_retries veces.
    """
    last_ai: AIMessage | None = None
    extra = extra_system_messages or []
    for _ in range(planner_config.max_retries + 1):
        msgs = [system_message] + extra + list(user_or_history_messages)
        ai_msg: AIMessage = planner_llm.invoke(msgs)
        last_ai = ai_msg
        if extract_tool_calls(ai_msg):
            break
    return last_ai  # type: ignore[return-value]


# ─────────────────────────────────────────────
# Helpers JSON para serializar salidas de tools
# ─────────────────────────────────────────────

def _json_default(obj: Any) -> Any:
    """
    Fallback para tipos no JSON-serializables (np.int64, sets, etc.).
    Mantiene estructura lo mejor posible en lugar de castear todo a str.
    """
    # Numpy genéricos → .item()
    try:
        import numpy as _np  # import local para no romper si no hay numpy
        if isinstance(obj, _np.generic):
            return obj.item()
    except Exception:
        pass

    # Sets → lista
    if isinstance(obj, (set, frozenset)):
        return list(obj)

    # Último recurso
    return str(obj)


# ─────────────────────────────────────────────
# 1) Utilidades: strip_think() + “último assistant real”
# ─────────────────────────────────────────────

_THINK_RE = re.compile(r"<think>.*?</think>\s*", flags=re.S | re.I)

def strip_think(txt: str) -> str:
    """Elimina <think>...</think> si existe (Qwen / Hermes), y recorta."""
    if not isinstance(txt, str):
        return ""
    return _THINK_RE.sub("", txt).strip()

def _is_pipeline_internal_ai(m: AnyMessage) -> bool:
    """
    Detecta mensajes internos del pipeline (summarizer/validator),
    para NO confundirlos con la respuesta real del LLM.
    """
    if not isinstance(m, AIMessage):
        return False

    addkw = getattr(m, "additional_kwargs", {}) or {}
    if addkw.get("pipeline_internal") is True:
        return True

    # Heurística por contenido (fallback defensivo)
    txt = _coerce_content_str(getattr(m, "content", "")).lstrip()
    if txt.startswith("## Resumen del pipeline"):
        return True
    if txt.startswith("## Resumen deep del pipeline"):
        return True
    if txt.startswith("### VALIDATOR"):
        return True

    return False

def find_last_assistant_real(messages: List[AnyMessage]) -> Optional[AIMessage]:
    """
    Devuelve el último AIMessage "real" (del LLM), ignorando mensajes internos del pipeline.
    """
    for m in reversed(messages or []):
        if isinstance(m, AIMessage) and not _is_pipeline_internal_ai(m):
            txt = _coerce_content_str(getattr(m, "content", "")).strip()
            if txt:
                return m
    return None


# ─────────────────────────────────────────────
# Summarizer helpers
# ─────────────────────────────────────────────

def _fmt_args(args: dict) -> str:
    if not args:
        return ""
    return ", ".join(f"{k}={repr(v)}" for k, v in args.items())


def _fmt_output(tool_name: str, v: Any) -> str:
    if isinstance(v, bool):
        return "Sí" if v else "No"

    # Embeddings → preview compacto
    if tool_name == "embed_texts":
        try:
            preview = []
            if isinstance(v, list):
                for idx, vec in enumerate(v):
                    if isinstance(vec, list):
                        preview.append(
                            {
                                "index": idx,
                                "dim": len(vec),
                                "head_5": vec[:5],
                            }
                        )
            return json.dumps(preview, ensure_ascii=False, indent=2)
        except Exception:
            return str(v)

    # Reranker → lista JSON bonita
    if tool_name == "rerank_qwen3":
        try:
            return json.dumps(v, ensure_ascii=False, indent=2)
        except Exception:
            return str(v)

    # Tablas / dicts grandes → JSON bonito
    if isinstance(v, (dict, list)):
        try:
            return json.dumps(v, ensure_ascii=False, indent=2, default=_json_default)
        except Exception:
            return str(v)

    return str(v)


def summarize_tool_runs(user_text: str, runs: List[Dict[str, Any]]) -> str:
    """
    Resumen user-friendly basado SOLO en las salidas de herramientas.
    Esto alimenta `summary.summarizer` y la sección dev "SUMMARIZER (basado en herramientas)".
    """
    if not runs:
        return (
            "No se invocó ninguna herramienta. "
            "No puedo responder con garantías a la pregunta sólo con razonamiento interno."
        )

    partes = [
        "📌 **Resumen basado en herramientas (sin alucinaciones)**",
    ]

    for r in runs:
        arg_str = _fmt_args(r["args"])
        out_str = _fmt_output(r["name"], r["output"])

        if r["name"] in (
            "embed_texts",
            "rerank_qwen3",
            "embed_context_tables",
            "semantic_search_in_csv",
            "judge_row_with_context",
        ):
            partes.append(
                f"- `{r['name']}({arg_str})`:\n\n```json\n{out_str}\n```"
            )
        else:
            partes.append(
                f"- `{r['name']}({arg_str})` → **{out_str}**"
            )

    return "\n".join(partes)


def build_user_answer(user_text: str, runs: List[Dict[str, Any]]) -> str:
    """
    Construye la respuesta 1:1 en lenguaje natural para el modo USER,
    usando EXCLUSIVAMENTE lo que viene en `runs` (tool-first, sin alucinaciones).

    Nota: es agnóstico, pero tiene atajos para algunas tools típicas
    (to_upper, is_palindrome, word_count, embed_texts, rerank_qwen3,
     y las tools de contexto: embed_context_tables,
     semantic_search_in_csv, judge_row_with_context).

    Si no reconoce nada, devolvemos "" y el caller
    hará fallback al resumen de herramientas.
    """
    if not runs:
        return ""

    sentences: List[str] = []

    # 1) Reranker: priorizamos porque suele ser la "respuesta clave"
    rr = next((r for r in runs if r.get("name") == "rerank_qwen3"), None)
    if rr is not None:
        out = rr.get("output")
        args = rr.get("args", {}) or {}
        docs = args.get("documents") or []

        if isinstance(out, list) and out:
            best = out[0]
            idx = best.get("index", 0)
            doc = best.get("document")
            score = best.get("score")

            # Si el documento no viene embebido, lo recuperamos de args.documents
            if doc is None and isinstance(docs, list) and isinstance(idx, int) and 0 <= idx < len(docs):
                doc = docs[idx]

            idx_h = idx + 1 if isinstance(idx, int) else idx

            if doc is not None:
                if isinstance(score, (int, float)):
                    sentences.append(
                        f"El documento más relevante es el #{idx_h}: \"{doc}\" "
                        f"(score ≈ {score:.3f})."
                    )
                else:
                    sentences.append(
                        f"El documento más relevante es el #{idx_h}: \"{doc}\"."
                    )

    # 2) to_upper
    for r in runs:
        if r.get("name") == "to_upper":
            txt = r.get("args", {}).get("text")
            out = r.get("output")
            if txt is not None and out is not None:
                sentences.append(f"He convertido \"{txt}\" a mayúsculas: **{out}**.")

    # 3) is_palindrome
    for r in runs:
        if r.get("name") == "is_palindrome":
            txt = r.get("args", {}).get("text")
            val = r.get("output")
            if txt:
                if bool(val):
                    sentences.append(f"\"{txt}\" es un palíndromo.")
                else:
                    sentences.append(f"\"{txt}\" no es un palíndromo.")

    # 4) word_count
    for r in runs:
        if r.get("name") == "word_count":
            txt = r.get("args", {}).get("text")
            n = r.get("output")
            if txt is not None and n is not None:
                sentences.append(f"El texto \"{txt}\" tiene {n} palabras.")

    # 5) embed_texts
    emb = next((r for r in runs if r.get("name") == "embed_texts"), None)
    if emb is not None:
        vecs = emb.get("output")
        n_vecs = len(vecs) if isinstance(vecs, list) else 0

        dim: Optional[int] = None
        if isinstance(vecs, list) and vecs:
            v0 = vecs[0]
            if isinstance(v0, list):  # lista cruda de floats
                dim = len(v0)
            elif isinstance(v0, dict) and "dim" in v0:
                try:
                    dim = int(v0["dim"])
                except Exception:
                    dim = None

        if n_vecs and dim:
            sentences.append(
                f"He generado embeddings de dimensión {dim} para {n_vecs} texto(s)."
            )
        elif n_vecs:
            sentences.append(
                f"He generado embeddings para {n_vecs} texto(s)."
            )

    # 6) embed_context_tables (tablas de parametrías / abreviaturas)
    for r in runs:
        if r.get("name") == "embed_context_tables":
            args = r.get("args", {}) or {}
            tables = args.get("table_paths") or args.get("tables") or []
            out = r.get("output")

            n_tables = len(tables) if isinstance(tables, list) else 0

            dim: Optional[int] = None
            if isinstance(out, dict):
                dim = out.get("embedding_dim") or out.get("dim")
                try:
                    if dim is not None:
                        dim = int(dim)
                except Exception:
                    dim = None

            if n_tables:
                if dim:
                    sentences.append(
                        f"He generado embeddings de dimensión {dim} para {n_tables} tabla(s) de contexto "
                        "(parametrías, diccionarios o definiciones)."
                    )
                else:
                    sentences.append(
                        f"He generado embeddings para {n_tables} tabla(s) de contexto "
                        "(parametrías, diccionarios o definiciones)."
                    )

    # 7) semantic_search_in_csv (búsqueda semántica en tablas / OCR tabular)
    for r in runs:
        if r.get("name") == "semantic_search_in_csv":
            args = r.get("args", {}) or {}
            query = args.get("query")
            csv_path = args.get("csv_path") or args.get("table_path")
            out = r.get("output")

            best_row = None
            if isinstance(out, list) and out:
                best_row = out[0]

            if best_row is not None:
                score = (
                    best_row.get("score")
                    if isinstance(best_row, dict)
                    else None
                )
                row_preview = best_row.get("row") if isinstance(best_row, dict) else None
                if isinstance(row_preview, dict):
                    row_preview = json.dumps(row_preview, ensure_ascii=False)
                if query and csv_path:
                    if isinstance(score, (int, float)):
                        sentences.append(
                            f"En la tabla `{csv_path}` he encontrado una fila muy relevante para la consulta "
                            f"\"{query}\" (score ≈ {score:.3f}). Fila ejemplo: {row_preview}."
                        )
                    else:
                        sentences.append(
                            f"En la tabla `{csv_path}` he encontrado una fila relevante para la consulta "
                            f"\"{query}\". Fila ejemplo: {row_preview}."
                        )

    # 8) judge_row_with_context (juicio de una fila de atributos vs parametrías/diccionarios)
    for r in runs:
        if r.get("name") == "judge_row_with_context":
            args = r.get("args", {}) or {}
            out = r.get("output") or {}

            contract_id = (
                out.get("contract_id")
                or out.get("row_id")
                or out.get("id")
                or args.get("contract_id")
                or args.get("row_id")
                or args.get("id")
            )
            judgement = (
                out.get("judgement")
                or out.get("verdict")
                or out.get("decision")
                or out.get("status")
            )
            reasons = out.get("reasons") or out.get("rule_hits") or out.get("details")

            if isinstance(reasons, list):
                reasons_str = "; ".join(map(str, reasons[:3]))
            else:
                reasons_str = str(reasons) if reasons is not None else ""

            if contract_id is not None and judgement is not None:
                sent = (
                    f"He evaluado la fila/contrato '{contract_id}' aplicando las tablas de parametrías "
                    f"y diccionarios de contexto. El juicio es: **{judgement}**."
                )
                if reasons_str:
                    sent += f" Motivos principales: {reasons_str}."
                sentences.append(sent)
            elif judgement is not None:
                sent = (
                    f"He evaluado la fila de atributos con las tablas de contexto; "
                    f"el juicio global es: **{judgement}**."
                )
                if reasons_str:
                    sent += f" Motivos principales: {reasons_str}."
                sentences.append(sent)

    if not sentences:
        # Fallback: si por lo que sea no pudimos mapear nada,
        # devolvemos cadena vacía y el caller decidirá usar el resumen de tools.
        return ""

    return " ".join(sentences)


# ─────────────────────────────────────────────
# Pequeños helpers de contexto (memoria / KB)
# ─────────────────────────────────────────────

def _format_memory_context(mem: Any) -> str:
    """
    Serializa el memory_context para pasarlo al planner como SystemMessage.

    Pensado para cosas tipo:
      - últimas N interacciones relevantes,
      - notas de usuario,
      - resúmenes de largo plazo.

    Mantenerlo breve es trabajo de memory.py; aquí sólo lo volcamos.
    """
    if not mem:
        return ""
    try:
        return json.dumps(mem, ensure_ascii=False, indent=2)
    except Exception:
        return str(mem)


def _format_kb_hint(kb_names: List[str]) -> str:
    if not kb_names:
        return ""
    return (
        "KBs disponibles para esta sesión:\n"
        + "\n".join(f"- {name}" for name in kb_names)
        + "\n\nPuedes decidir llamar a herramientas que lean o crucen estas KBs "
          "si es necesario (por ejemplo, comparar filas de una tabla con una tabla "
          "de parámetros / reglas de calidad y emitir una tabla de juicios)."
    )


# ─────────────────────────────────────────────
# Builder del grafo LangGraph
# ─────────────────────────────────────────────

def build_graph_agent(
    planner_llm,
    tools: List[Any],
    planner_config: PlannerConfig | None = None,
):
    """
    Grafo:

        START → ANALYZER → PLANNER
                      ├─(tool_calls)→ EXECUTOR → CATCHER → SUMMARIZER → VALIDATOR → END
                      └─────────────→ SUMMARIZER → VALIDATOR → END
    """
    cfg = planner_config or PlannerConfig()
    system_msg = build_planner_system_message(cfg)

    # ANALYZER (rule-based inicial, pero ya guarda payload rico)
    def analyzer_node(state: State) -> Dict[str, Any]:
        messages = state.get("messages", [])
        user_messages = [m for m in messages if isinstance(m, HumanMessage)]
        last_user = user_messages[-1] if user_messages else None
        user_text = last_user.content if isinstance(last_user, HumanMessage) else ""

        # Permitimos que el llamador ya haya rellenado user_prompt
        user_prompt = state.get("user_prompt") or user_text

        # Input payload más rico: aquí podríamos meter referencias a tablas/JSON.
        input_payload: Dict[str, Any] = {
            "user_prompt": user_prompt,
        }

        # Split sencillo multi-sentencia: puntos y saltos de línea
        raw = str(user_prompt).replace("\n", " ")
        subqueries: List[str] = []
        for part in raw.split("."):
            part = part.strip()
            if part:
                subqueries.append(part)
        if not subqueries and user_prompt:
            subqueries = [user_prompt]

        subqueries_logic = [f"q{i+1}" for i in range(len(subqueries))]
        propositional_logic = " ∧ ".join(subqueries_logic) if subqueries_logic else ""

        analyzer: AnalyzerResult = {
            "input_payload": input_payload,
            "propositional_logic": propositional_logic,
            "subqueries": subqueries,
            "subqueries_logic": subqueries_logic,
        }

        return {"analyzer": analyzer}

    # PLANNER (ve memoria y KB names como contextos)
    def planner_node(state: State) -> Dict[str, Any]:
        msgs: List[AnyMessage] = state["messages"]

        # Contextos adicionales
        mem_ctx = state.get("memory_context")
        kb_names = state.get("kb_names") or []

        mem_str = _format_memory_context(mem_ctx)
        kb_str = _format_kb_hint(kb_names)

        extra_system_messages: List[SystemMessage] = []
        if mem_str:
            extra_system_messages.append(
                SystemMessage(
                    content=(
                        "Contexto de memoria para esta sesión "
                        "(puede contener preferencias, interacciones previas o notas):\n"
                        f"{mem_str}"
                    )
                )
            )
        if kb_str:
            extra_system_messages.append(SystemMessage(content=kb_str))

        ai_msg: AIMessage = call_planner_with_retry(
            planner_llm=planner_llm,
            system_message=system_msg,
            user_or_history_messages=msgs,
            planner_config=cfg,
            extra_system_messages=extra_system_messages,
        )

        # 5) Invariante: guardamos raw/clean del LLM (sirve para modo sin tools)
        llm_raw_out = _coerce_content_str(getattr(ai_msg, "content", ""))
        llm_clean_out = strip_think(llm_raw_out)

        tool_calls = extract_tool_calls(ai_msg)
        analyzer = state.get("analyzer") or {}
        subqs: List[str] = analyzer.get("subqueries") or []

        if not subqs:
            user_messages = [m for m in msgs if isinstance(m, HumanMessage)]
            last_user = user_messages[-1] if user_messages else None
            if isinstance(last_user, HumanMessage):
                subqs = [last_user.content]

        plan_trajs: List[PlannerTrajectory] = []
        if subqs:
            desc_lines: List[str] = []
            if not tool_calls:
                desc_lines.append(
                    "No se planificó ninguna llamada a herramientas; "
                    "el agente responderá directamente."
                )
            else:
                for idx, tc in enumerate(tool_calls, start=1):
                    desc_lines.append(
                        f"Paso {idx}: llamar a la herramienta `{tc['name']}` "
                        f"con args={tc.get('args', {})}."
                    )
            plan_trajs.append(
                PlannerTrajectory(
                    subquery=subqs[0],
                    description="\n".join(desc_lines),
                )
            )

        return {
            "messages": [ai_msg],
            "planner_trajs": plan_trajs,
            "llm_raw_out": llm_raw_out,
            "llm_clean_out": llm_clean_out,
        }

    # EXECUTOR
    def executor_node(state: State) -> Dict[str, Any]:
        messages = state["messages"]
        ai_msgs = [m for m in messages if isinstance(m, AIMessage)]
        if not ai_msgs:
            return {"messages": [], "executor_steps": []}

        ai_plan = ai_msgs[-1]
        tool_calls = extract_tool_calls(ai_plan)
        if not tool_calls:
            return {"messages": [], "executor_steps": []}

        tool_msgs: List[ToolMessage] = []
        exec_steps: List[ExecutorStep] = []

        for tc in tool_calls:
            name = tc["name"]
            args = tc.get("args", {}) or {}

            try:
                tool_obj = next(t for t in tools if t.name == name)
                observation = tool_obj.invoke(args)
            except StopIteration:
                observation = {"error": f"Tool '{name}' no encontrada."}
            except Exception as e:
                observation = {"error": f"Excepción ejecutando tool '{name}': {e!r}"}

            try:
                payload = json.dumps(
                    {"value": observation},
                    ensure_ascii=False,
                    default=_json_default,
                )
            except TypeError:
                payload = json.dumps(
                    {"value": str(observation)},
                    ensure_ascii=False,
                )

            tool_call_id = tc["id"]

            tool_msgs.append(
                ToolMessage(
                    content=payload,
                    tool_call_id=tool_call_id,
                )
            )

            exec_steps.append(
                ExecutorStep(
                    tool_call_id=tool_call_id,
                    tool_name=name,
                    args=args,
                )
            )

        return {
            "messages": tool_msgs,
            "executor_steps": exec_steps,
        }

    # CATCHER
    def catcher_node(state: State) -> Dict[str, Any]:
        messages = state["messages"]

        ai_msgs = [m for m in messages if isinstance(m, AIMessage)]
        ai_plan = next(
            (m for m in reversed(ai_msgs) if extract_tool_calls(m)),
            None,
        )
        tool_calls = extract_tool_calls(ai_plan) if ai_plan else []

        tmsgs: List[ToolMessage] = [m for m in messages if isinstance(m, ToolMessage)]

        runs: List[Dict[str, Any]] = []
        for tc in tool_calls:
            tm = next((t for t in tmsgs if t.tool_call_id == tc["id"]), None)
            if tm is None:
                continue
            raw = tm.content
            try:
                decoded = json.loads(raw)
                output = decoded.get("value", decoded)
            except Exception:
                output = raw
            runs.append(
                {
                    "id": tc["id"],
                    "name": tc["name"],
                    "args": tc.get("args", {}) or {},
                    "output": output,
                }
            )

        return {"tool_runs": runs}

    # SUMMARIZER
    def summarizer_node(state: State) -> Dict[str, Any]:
        messages = state["messages"]
        user_messages = [m for m in messages if isinstance(m, HumanMessage)]
        last_user = user_messages[-1] if user_messages else None
        user_text = last_user.content if isinstance(last_user, HumanMessage) else ""
        user_prompt = state.get("user_prompt") or user_text

        runs = state.get("tool_runs", []) or []

        # 2) Parche SUMMARIZER (regla de oro):
        # Si NO hay tools (runs vacío) y el último AI NO tiene tool_calls,
        # user_out debe ser la salida directa del LLM (limpia de <think>).
        if not runs:
            last_ai = find_last_assistant_real(messages)
            last_ai_has_tools = bool(extract_tool_calls(last_ai)) if last_ai else False

            llm_raw = state.get("llm_raw_out") or (_coerce_content_str(getattr(last_ai, "content", "")) if last_ai else "")
            llm_clean = state.get("llm_clean_out") or strip_think(llm_raw)

            if last_ai_has_tools:
                # Caso raro: se planearon tools pero no hay runs (falló executor/catcher).
                user_answer = (
                    "Se planificaron llamadas a herramientas, pero no se obtuvo ninguna salida. "
                    "Revisa EXECUTOR/CATCHER o el registro de tools."
                )
            else:
                user_answer = llm_clean or "¿Qué te gustaría hacer?"

            tools_summary_text = summarize_tool_runs(user_prompt, runs)

            analyzer = state.get("analyzer") or {}
            subqs = analyzer.get("subqueries") or []
            logic = analyzer.get("propositional_logic") or ""
            input_payload = analyzer.get("input_payload") or {}

            if analyzer:
                analyzer_text_lines = [
                    f"Input payload: {input_payload!r}",
                    f"Lógica proposicional: {logic or '(no construida)'}",
                    f"Subconsultas ({len(subqs)}):",
                ]
                for s in subqs:
                    analyzer_text_lines.append(f"- {s}")
                analyzer_text = "\n".join(analyzer_text_lines)
            else:
                analyzer_text = "No se ejecutó ANALYZER o no dejó estado."

            planner_trajs = state.get("planner_trajs", []) or []
            if planner_trajs:
                pl_lines: List[str] = []
                for i, tr in enumerate(planner_trajs, start=1):
                    pl_lines.append(f"Subquery {i}: {tr.get('subquery', '')}")
                    desc = tr.get("description")
                    if desc:
                        pl_lines.append(desc)
                planner_text = "\n".join(pl_lines)
            else:
                planner_text = (
                    "No se construyó un plan de herramientas; probablemente se respondió "
                    "directamente (o no hubo tool_calls)."
                )

            executor_steps = state.get("executor_steps", []) or []
            if executor_steps:
                ex_lines: List[str] = [
                    f"Se ejecutaron {len(executor_steps)} llamadas a herramientas:"
                ]
                for step in executor_steps:
                    ex_lines.append(
                        f"- tool_call_id={step['tool_call_id']}, "
                        f"name={step['tool_name']}, args={step['args']!r}"
                    )
                executor_text = "\n".join(ex_lines)
            else:
                executor_text = "No se ejecutó ninguna herramienta para esta consulta."

            catcher_text = "Catcher no encontró resultados de tools (runs vacío)."

            # summarizer_text para DEV/DEEP: en modo sin tools, dejamos constancia
            summarizer_text = "No se invocaron herramientas. Respuesta directa del modelo (passthrough)."

            summary_dict: SummaryDict = SummaryDict(
                analyzer=analyzer_text,
                planner=planner_text,
                executor=executor_text,
                catcher=catcher_text,
                summarizer=summarizer_text,
                final_answer=user_answer,
            )

            sections = [
                "## Resumen del pipeline",
                "### ANALYZER",
                analyzer_text,
                "### PLANNER",
                planner_text,
                "### EXECUTOR",
                executor_text,
                "### CATCHER",
                catcher_text,
                "### SUMMARIZER (basado en herramientas)",
                tools_summary_text,
                "### RESPUESTA FINAL (modo usuario)",
                user_answer,
            ]
            answer_markdown = "\n\n".join(sections)

            final_ai = AIMessage(
                content=answer_markdown,
                additional_kwargs={"pipeline_internal": True, "node": "summarizer"},
            )

            dev_out = answer_markdown
            deep_out = "\n\n".join([
                "## Resumen deep del pipeline",
                "### ANALYZER",
                analyzer_text,
                "### PLANNER",
                planner_text,
                "### EXECUTOR",
                executor_text,
                "### CATCHER",
                catcher_text,
                "### SUMMARIZER",
                summarizer_text,
                "### RESPUESTA FINAL",
                user_answer,
            ])
            user_out = user_answer

            return {
                "messages": [final_ai],
                "summary": summary_dict,
                "pipeline_summary": summary_dict,
                "dev_out": dev_out,
                "deep_out": deep_out,
                "user_out": user_out,
            }

        tools_summary_text = summarize_tool_runs(user_prompt, runs)

        analyzer = state.get("analyzer") or {}
        subqs = analyzer.get("subqueries") or []
        logic = analyzer.get("propositional_logic") or ""
        input_payload = analyzer.get("input_payload") or {}

        if analyzer:
            analyzer_text_lines = [
                f"Input payload: {input_payload!r}",
                f"Lógica proposicional: {logic or '(no construida)'}",
                f"Subconsultas ({len(subqs)}):",
            ]
            for s in subqs:
                analyzer_text_lines.append(f"- {s}")
            analyzer_text = "\n".join(analyzer_text_lines)
        else:
            analyzer_text = "No se ejecutó ANALYZER o no dejó estado."

        planner_trajs = state.get("planner_trajs", []) or []
        if planner_trajs:
            pl_lines: List[str] = []
            for i, tr in enumerate(planner_trajs, start=1):
                pl_lines.append(f"Subquery {i}: {tr.get('subquery', '')}")
                desc = tr.get("description")
                if desc:
                    pl_lines.append(desc)
            planner_text = "\n".join(pl_lines)
        else:
            planner_text = (
                "No se construyó un plan de herramientas; probablemente se respondió "
                "directamente (o no hubo tool_calls)."
            )

        executor_steps = state.get("executor_steps", []) or []
        if executor_steps:
            ex_lines: List[str] = [
                f"Se ejecutaron {len(executor_steps)} llamadas a herramientas:"
            ]
            for step in executor_steps:
                ex_lines.append(
                    f"- tool_call_id={step['tool_call_id']}, "
                    f"name={step['tool_name']}, args={step['args']!r}"
                )
            executor_text = "\n".join(ex_lines)
        else:
            executor_text = "No se ejecutó ninguna herramienta para esta consulta."

        if runs:
            ca_lines: List[str] = [
                f"Catcher recopiló {len(runs)} resultados de tools."
            ]
            for r in runs:
                ca_lines.append(
                    f"- {r['name']}({r['args']!r}) → output tipo {type(r['output']).__name__}"
                )
            catcher_text = "\n".join(ca_lines)
        else:
            catcher_text = "Catcher no encontró resultados de tools (runs vacío)."

        # Resumen tool-based (para DEV/DEEP)
        summarizer_text = tools_summary_text

        # Respuesta 1:1 en lenguaje natural para USER (tool-first)
        user_answer = build_user_answer(user_prompt, runs)
        if not user_answer:
            # Fallback conservador: si por lo que sea no pudimos mapear nada,
            # devolvemos el resumen de herramientas como antes.
            user_answer = tools_summary_text

        summary_dict: SummaryDict = SummaryDict(
            analyzer=analyzer_text,
            planner=planner_text,
            executor=executor_text,
            catcher=catcher_text,
            summarizer=summarizer_text,
            final_answer=user_answer,
        )

        # Esta respuesta (answer_markdown) es la vista "dev" con todo el pipeline.
        sections = [
            "## Resumen del pipeline",
            "### ANALYZER",
            analyzer_text,
            "### PLANNER",
            planner_text,
            "### EXECUTOR",
            executor_text,
            "### CATCHER",
            catcher_text,
            "### SUMMARIZER (basado en herramientas)",
            summarizer_text,
            "### RESPUESTA FINAL (modo usuario)",
            user_answer,
        ]
        answer_markdown = "\n\n".join(sections)

        final_ai = AIMessage(
            content=answer_markdown,
            additional_kwargs={"pipeline_internal": True, "node": "summarizer"},
        )

        # Además rellenamos dev_out / deep_out / user_out:
        dev_out = answer_markdown
        deep_out = "\n\n".join([
            "## Resumen deep del pipeline",
            "### ANALYZER",
            analyzer_text,
            "### PLANNER",
            planner_text,
            "### EXECUTOR",
            executor_text,
            "### CATCHER",
            catcher_text,
            "### SUMMARIZER",
            summarizer_text,
            "### RESPUESTA FINAL",
            user_answer,
        ])
        user_out = user_answer

        return {
            "messages": [final_ai],
            "summary": summary_dict,
            "pipeline_summary": summary_dict,
            "dev_out": dev_out,
            "deep_out": deep_out,
            "user_out": user_out,
        }

    # VALIDATOR (heurística simple, preparada para LLM en el futuro)
    def validator_node(state: State) -> Dict[str, Any]:
        """
        Pequeño validador que mira:
          - si hubo tools,
          - si el Summarizer dijo "no se ejecutó ninguna herramienta",
          - si el final_answer está vacío,
          - y heurísticas ligeras sobre prompts tabulares/contratos.

        Marca all_covered=False en casos sospechosos.
        Más adelante se puede reemplazar por un LLM que reciba:
          (user_prompt, tool_runs, final_answer) y devuelva ValidatorResult.
        """
        user_prompt = state.get("user_prompt") or ""
        summary = state.get("pipeline_summary") or state.get("summary") or {}
        final_answer = summary.get("final_answer") or ""
        summarizer_text = summary.get("summarizer") or ""
        runs = state.get("tool_runs", []) or []

        # 3) Guardrail en VALIDATOR: auto-reparación (modo sin tools)
        bad_templates = (
            "no se invocó ninguna herramienta",
            "no puedo responder con garantías",
            "sin herramientas no puedo",
        )
        if runs == [] and any(t in final_answer.strip().lower() for t in bad_templates):
            last_ai = find_last_assistant_real(state.get("messages", []) or [])
            raw = state.get("llm_raw_out") or (_coerce_content_str(getattr(last_ai, "content", "")) if last_ai else "")
            direct = state.get("llm_clean_out") or strip_think(raw)

            if direct:
                final_answer = direct
                try:
                    summary["final_answer"] = direct
                except Exception:
                    pass

                # también reparamos user_out si estaba “apagado”
                state["user_out"] = direct

        all_covered = True
        reasons: List[str] = []

        if not final_answer.strip():
            all_covered = False
            reasons.append("La respuesta final está vacía.")

        if "No se invocó ninguna herramienta" in summarizer_text and runs:
            all_covered = False
            reasons.append(
                "Inconsistencia: el SUMMARIZER dice que no hubo tools, "
                "pero tool_runs no está vacío."
            )

        # Heurística: prompt tabular sin tools
        if runs == [] and user_prompt:
            if any(tok in user_prompt.lower() for tok in ["tabla", "table", "fila", "row", "columna", "column"]):
                all_covered = False
                reasons.append(
                    "El usuario menciona estructuras tabulares pero no se invocaron herramientas; "
                    "puede faltar cálculo determinista sobre tablas/diccionarios."
                )

        # Heurística adicional: caso de contratos sin juicio explícito
        if "contrato" in user_prompt.lower() or "contract" in user_prompt.lower():
            has_judge = any(r.get("name") == "judge_row_with_context" for r in runs)
            if not has_judge:
                all_covered = False
                reasons.append(
                    "El usuario menciona contratos pero no se detectó ninguna ejecución "
                    "de `judge_row_with_context`; podría faltar el juicio fila+contexto."
                )

        if not reasons and all_covered:
            reasons.append("No se detectaron problemas obvios de cobertura.")

        validator: ValidatorResult = {
            "all_covered": all_covered,
            "reasoning": "\n".join(reasons),
        }

        # Mensaje para la traza dev
        validator_msg = AIMessage(
            content=(
                "### VALIDATOR\n\n"
                f"- all_covered: {all_covered}\n"
                f"- reasoning:\n{validator['reasoning']}"
            ),
            additional_kwargs={"pipeline_internal": True, "node": "validator"},
        )

        return {
            "validator": validator,
            "messages": [validator_msg],
            "pipeline_summary": summary,
            "summary": summary,
            "user_out": final_answer if isinstance(final_answer, str) and final_answer.strip() else state.get("user_out"),
        }

    # Router
    def route_from_planner(state: State) -> str:
        messages = state["messages"]
        ai_msgs = [m for m in messages if isinstance(m, AIMessage)]
        if not ai_msgs:
            return "summarizer"

        last_ai = ai_msgs[-1]
        if extract_tool_calls(last_ai):
            return "executor"
        return "summarizer"

    # Build graph
    builder = StateGraph(State)

    builder.add_node("analyzer", analyzer_node)
    builder.add_node("planner", planner_node)
    builder.add_node("executor", executor_node)
    builder.add_node("catcher", catcher_node)
    builder.add_node("summarizer", summarizer_node)
    builder.add_node("validator", validator_node)

    builder.add_edge(START, "analyzer")
    builder.add_edge("analyzer", "planner")
    builder.add_conditional_edges(
        "planner",
        route_from_planner,
        ["executor", "summarizer"],
    )
    builder.add_edge("executor", "catcher")
    builder.add_edge("catcher", "summarizer")
    builder.add_edge("summarizer", "validator")
    builder.add_edge("validator", END)

    graph_app = builder.compile()
    return graph_app


# ─────────────────────────────────────────────
# Logic loader (registro de grafos)
# ─────────────────────────────────────────────

@dataclass
class LogicConfig:
    module: str = "agnostic_agent.logic"
    builder_fn: str = "build_graph_agent"


def load_logic(
    planner_llm: Any,
    tools: List[Any],
    planner_config: Optional[PlannerConfig] = None,
    logic_config: Optional[LogicConfig] = None,
) -> Any:
    """
    Carga y ejecuta la función builder que construye el grafo del agente.

    Por defecto usa este mismo módulo:
        agnostic_agent.logic.build_graph_agent
    """
    cfg = logic_config or LogicConfig()

    if cfg.module == "agnostic_agent.logic":
        builder: Callable[..., Any] = globals().get(cfg.builder_fn)  # type: ignore[assignment]
        if builder is None or not callable(builder):
            raise AttributeError(
                f"No se encontró función builder '{cfg.builder_fn}' en agnostic_agent.logic."
            )
        return builder(planner_llm, tools, planner_config)

    import importlib

    try:
        mod = importlib.import_module(cfg.module)
    except ModuleNotFoundError as e:
        raise ImportError(
            f"No se pudo importar el módulo de lógica '{cfg.module}'."
        ) from e

    builder = getattr(mod, cfg.builder_fn, None)
    if builder is None or not callable(builder):
        raise AttributeError(
            f"El módulo '{cfg.module}' no tiene una función callable '{cfg.builder_fn}'."
        )

    return builder(planner_llm, tools, planner_config)


In [ ]:
#@title agnostic_agent/agent.py
%%writefile agnostic_agent/agent.py
from __future__ import annotations

from typing import Any, Dict, List, Optional, Union, Tuple
from pathlib import Path
import os

import yaml
from langchain_core.messages import (
    HumanMessage,
    AIMessage,
    ToolMessage,
    AnyMessage,
)

from .capabilities import PlannerConfig, build_planner_llm
from .logic import load_logic, State
from .communication import (
    AgentInput,
    AgentOutput,
    AgentView,
    AgentSummary,
    ToolRun,
)
from .tools import get_default_tools  # ✅ catálogo global de tools
from .memory import read_memory, write_memory  # ✅ memoria multi-nivel (in-memory)
from .context import (  # ✅ KBs externas/tabulares
    KnowledgeBase,
    get_default_context,
    get_kb_by_names,
    build_kb_from_setup,
)


class Agent:
    """
    Agente agnóstico sobre LangGraph + Qwen3.

    Patrones de inicialización:

        # 1) Totalmente por defecto (sin setup.yaml)
        agent = Agent.init()

        # 2) Pasando path a setup.yaml (panel de control middleware)
        agent = Agent.init("setup.yaml")
        # o bien:
        agent = Agent.init(setup_path="setup.yaml")

        # 3) Pasando un PlannerConfig explícito (ignora planner de setup.yaml)
        agent = Agent.init(PlannerConfig(temperature=0.0))

        # 4) Pasando tablas de contexto (parametrías / abreviaturas, etc.)
        agent = Agent.init(
            "setup.yaml",
            context_tables=[
                "/content/parametrias.csv",
                "/content/diccionario_abreviaturas.csv",
            ],
        )

    - run_turn(...) SIEMPRE devuelve un dict:

        {
          "dev_out":  {...},  # traza completa (pipeline + raw_state)
          "deep_out": {...},  # resumen por sección (ANALYZER/PLANNER/...).
          "user_out": {...},  # respuesta 1:1 basada en herramientas.
        }

    El grafo deja en el estado:
      - tool_runs
      - summary / pipeline_summary (SummaryDict)
      - user_out / deep_out / dev_out (strings opcionales)
    y este wrapper los empaqueta en AgentOutput.

    Además:
      - Resuelve session_id / kb_names a partir de AgentInput.
      - Inyecta memory_context (read_memory) en el estado.
      - Inyecta knowledge_bases y context_tables (parametrías, abreviaturas, etc.).
      - Al final de cada turno actualiza la memoria con write_memory.
    """

    def __init__(
        self,
        graph_app: Any,
        planner_config: PlannerConfig,
        tools: List[Any],
        *,
        setup_path: Optional[str] = None,
        setup_config: Optional[Dict[str, Any]] = None,
        memory_cfg: Optional[Dict[str, Any]] = None,
        knowledge_bases: Optional[List[KnowledgeBase]] = None,
        context_tables: Optional[List[str]] = None,
        context_cfg: Optional[Dict[str, Any]] = None,
    ) -> None:
        self.graph_app = graph_app
        self.planner_config = planner_config
        self.tools = tools

        # Panel de control cargado desde setup.yaml (si existe)
        self.setup_path: Optional[str] = setup_path
        self.setup_config: Dict[str, Any] = setup_config or {}

        # Config de memoria (session / short_term / long_term)
        self.memory_cfg: Dict[str, Any] = memory_cfg or {}

        # KBs registradas (tabulares, vectores, APIs, etc.)
        self.knowledge_bases: List[KnowledgeBase] = (
            knowledge_bases if knowledge_bases is not None else get_default_context()
        )

        # Tablas CSV de contexto (parametrías, abreviaturas/definiciones, etc.)
        self.context_tables: List[str] = context_tables or []

        # Config de contexto crudo desde setup.yaml (opcional)
        self.context_cfg: Dict[str, Any] = context_cfg or {}

        # Estado de conversación multi-turn (historial reducido)
        self._state: Optional[Dict[str, Any]] = None

    # ------------------------------------------------------------------
    # Helpers de setup.yaml
    # ------------------------------------------------------------------
    @staticmethod
    def _load_setup_config(
        setup_path: Optional[Union[str, Path]],
    ) -> Tuple[Optional[Path], Dict[str, Any]]:
        """
        Intenta cargar setup.yaml (o el path que se pase).

        Orden de resolución:
          1) setup_path explícito (argumento).
          2) AGENT_SETUP_PATH en variables de entorno.
        """
        cfg: Dict[str, Any] = {}

        path_obj: Optional[Path] = None
        if isinstance(setup_path, (str, Path)):
            path_obj = Path(setup_path)
        else:
            env_path = os.getenv("AGENT_SETUP_PATH")
            if env_path:
                path_obj = Path(env_path)

        if path_obj is None:
            return None, cfg

        if not path_obj.is_file():
            # No levantamos excepción para no romper en Colab si no está el archivo
            print(f"[Agent] ⚠️ setup.yaml no encontrado en: {path_obj}. Se usarán defaults.")
            return path_obj, cfg

        try:
            with path_obj.open("r", encoding="utf-8") as f:
                data = yaml.safe_load(f) or {}
            if not isinstance(data, dict):
                print(f"[Agent] ⚠️ setup.yaml no tiene formato dict en {path_obj}. Ignorando.")
                return path_obj, {}
            cfg = data
        except Exception as e:
            print(f"[Agent] ⚠️ Error leyendo setup.yaml ({path_obj}): {e!r}")
            cfg = {}

        return path_obj, cfg

    @staticmethod
    def _apply_model_env_from_setup(setup_cfg: Dict[str, Any]) -> None:
        """
        Opcional: aplica variables de entorno para modelos / endpoints
        a partir de setup.yaml, sin pisar lo que ya esté definido.
        """
        models_cfg = setup_cfg.get("models") or {}

        # LLM
        llm_cfg = models_cfg.get("llm") or {}
        llm_api_base = llm_cfg.get("api_base")
        llm_served_name = llm_cfg.get("served_name")

        if llm_api_base and "VLLM_API_BASE" not in os.environ:
            os.environ["VLLM_API_BASE"] = str(llm_api_base)
        if llm_api_base and "VLLM_LLM_API_BASE" not in os.environ:
            os.environ["VLLM_LLM_API_BASE"] = str(llm_api_base)
        if llm_served_name and "LLM_SERVED_NAME" not in os.environ:
            os.environ["LLM_SERVED_NAME"] = str(llm_served_name)

        # Embeddings
        emb_cfg = models_cfg.get("emb") or {}
        emb_api_base = emb_cfg.get("api_base")
        emb_served_name = emb_cfg.get("served_name")

        if emb_api_base and "VLLM_EMB_API_BASE" not in os.environ:
            os.environ["VLLM_EMB_API_BASE"] = str(emb_api_base)
        if emb_served_name and "EMB_SERVED_NAME" not in os.environ:
            os.environ["EMB_SERVED_NAME"] = str(emb_served_name)

        # Reranker
        rerank_cfg = models_cfg.get("rerank") or {}
        rerank_api_base = rerank_cfg.get("api_base")
        rerank_served_name = rerank_cfg.get("served_name")

        if rerank_api_base and "VLLM_RERANK_API_BASE" not in os.environ:
            os.environ["VLLM_RERANK_API_BASE"] = str(rerank_api_base)
        if rerank_served_name and "RERANK_SERVED_NAME" not in os.environ:
            os.environ["RERANK_SERVED_NAME"] = str(rerank_served_name)

        # Clave dummy para OpenAI-compatible (vLLM la ignora pero la requiere)
        if "OPENAI_API_KEY" not in os.environ:
            os.environ["OPENAI_API_KEY"] = "EMPTY"

    @staticmethod
    def _build_kb_from_setup(setup_cfg: Dict[str, Any]) -> List[KnowledgeBase]:
        """
        Construye la lista de KnowledgeBase a partir de setup.yaml usando
        el helper genérico de context.build_kb_from_setup.

        Si no hay nada en el YAML, cae en get_default_context().
        """
        kb_list = build_kb_from_setup(setup_cfg)
        if not kb_list:
            kb_list = get_default_context()
        return kb_list

    @staticmethod
    def _resolve_context_tables(
        setup_cfg: Dict[str, Any],
        explicit_context_tables: Optional[List[str]],
    ) -> Tuple[List[str], Dict[str, Any]]:
        """
        Resuelve tablas de contexto (CSV) y config de contexto.

        Prioridad:
          1) context_tables explícito pasado a Agent.init(...)
          2) setup.yaml:
               context:
                 tables: [...]
               # o bien, compat:
               context_tables: [...]
        """
        context_cfg: Dict[str, Any] = setup_cfg.get("context") or {}

        yaml_tables = context_cfg.get("tables") or setup_cfg.get("context_tables") or []
        if isinstance(yaml_tables, str):
            yaml_tables = [yaml_tables]
        yaml_tables = [str(p) for p in yaml_tables] if isinstance(yaml_tables, list) else []

        if explicit_context_tables is not None:
            final_tables = list(explicit_context_tables)
        else:
            final_tables = yaml_tables

        return final_tables, context_cfg

    # ------------------------------------------------------------------
    # Factory
    # ------------------------------------------------------------------
    @classmethod
    def init(
        cls,
        config_or_setup: Optional[Union[PlannerConfig, str, os.PathLike]] = None,
        tools: Optional[List[Any]] = None,
        *,
        context_tables: Optional[List[str]] = None,
    ) -> "Agent":
        """
        Construye un Agent listo para usar.

        Patrones soportados:

            agent = Agent.init()
            agent = Agent.init("setup.yaml")
            agent = Agent.init(setup_path)  # PathLike
            agent = Agent.init(PlannerConfig(...))
            agent = Agent.init("setup.yaml", context_tables=[...])

        NOTA:
        - Si pasas PlannerConfig → se ignora el bloque `planner` de setup.yaml.
        - Si quieres combinar ambas cosas, construye tú el PlannerConfig
          leyendo el YAML y pásalo aquí.
        """
        # 1) Resolver si el primer parámetro es un PlannerConfig o un path
        setup_path: Optional[Union[str, Path]] = None
        planner_cfg: Optional[PlannerConfig] = None

        if isinstance(config_or_setup, PlannerConfig):
            planner_cfg = config_or_setup
        elif isinstance(config_or_setup, (str, os.PathLike)):
            setup_path = config_or_setup

        # 2) Cargar setup.yaml (si existe)
        setup_path_resolved, setup_cfg = cls._load_setup_config(setup_path)

        # 3) Aplicar envs de modelos (si vienen en setup.yaml)
        if setup_cfg:
            cls._apply_model_env_from_setup(setup_cfg)

        # 4) PlannerConfig: o bien el explícito, o bien override desde YAML
        if planner_cfg is not None:
            cfg = planner_cfg
        else:
            cfg = PlannerConfig()
            planner_section = setup_cfg.get("planner") or {}
            # Sobrescribimos sólo campos conocidos (tolerante)
            for key, value in planner_section.items():
                if hasattr(cfg, key):
                    setattr(cfg, key, value)

        # 5) Tools: prioridad al parámetro explícito; si no, leer de setup.yaml
        if tools is not None:
            tools_list = tools
        else:
            tools_section = setup_cfg.get("tools") or {}
            enabled_names = tools_section.get("enabled")
            tools_list = get_default_tools(enabled_names=enabled_names)

        # 6) LLM planner bindeado a las tools
        planner_llm = build_planner_llm(cfg)
        planner_llm = planner_llm.bind_tools(tools_list)

        # 7) Construir grafo principal
        graph_app = load_logic(
            planner_llm=planner_llm,
            tools=tools_list,
            planner_config=cfg,
        )

        # 8) Config de memoria desde setup.yaml (si existe)
        memory_cfg = setup_cfg.get("memory") or {}

        # 9) Knowledge Bases (tabulares / vectores / APIs) desde setup.yaml
        kb_list = cls._build_kb_from_setup(setup_cfg)

        # 10) Tablas de contexto (parametrías, abreviaturas, etc.)
        final_context_tables, context_cfg = cls._resolve_context_tables(
            setup_cfg=setup_cfg,
            explicit_context_tables=context_tables,
        )

        return cls(
            graph_app=graph_app,
            planner_config=cfg,
            tools=tools_list,
            setup_path=str(setup_path_resolved) if setup_path_resolved else None,
            setup_config=setup_cfg,
            memory_cfg=memory_cfg,
            knowledge_bases=kb_list,
            context_tables=final_context_tables,
            context_cfg=context_cfg,
        )

    # ------------------------------------------------------------------
    # Helpers internos
    # ------------------------------------------------------------------
    def _coerce_input(
        self,
        user_input: Union[str, Dict[str, Any], AgentInput],
    ) -> AgentInput:
        """
        Normaliza la entrada a AgentInput:

        - str  -> AgentInput(user_prompt=...)
        - dict -> AgentInput(**dict)
        - AgentInput -> se respeta tal cual
        """
        if isinstance(user_input, AgentInput):
            return user_input
        if isinstance(user_input, dict):
            return AgentInput(**user_input)
        return AgentInput(user_prompt=str(user_input))

    def _clean_prev_messages(self) -> List[AnyMessage]:
        """
        Limpia el historial previo para evitar ToolMessages gigantes en turnos futuros.
        """
        msgs: List[AnyMessage] = []
        if self._state is None:
            return msgs

        for m in self._state.get("messages", []):
            # ❌ No arrastramos ToolMessages con JSONs enormes
            if isinstance(m, ToolMessage):
                continue
            msgs.append(m)
        return msgs

    def _build_deep_text(
        self,
        summary_obj: Optional[AgentSummary],
    ) -> str:
        """
        Construye la vista 'deep' (resumen por sección) a partir de AgentSummary.

        - No incluye el raw_state completo.
        - Es más compacta que la traza dev_out, pero sigue seccionada.
        """
        if summary_obj is None:
            # Si por alguna razón no hay summary, devolvemos cadena vacía;
            # el caller puede hacer fallback.
            return ""

        parts: List[str] = ["## Resumen deep del pipeline"]

        if summary_obj.analyzer:
            parts.append("### ANALYZER\n" + summary_obj.analyzer)
        if summary_obj.planner:
            parts.append("### PLANNER\n" + summary_obj.planner)
        if summary_obj.executor:
            parts.append("### EXECUTOR\n" + summary_obj.executor)
        if summary_obj.catcher:
            parts.append("### CATCHER\n" + summary_obj.catcher)
        if summary_obj.summarizer:
            parts.append("### SUMMARIZER (basado en herramientas)\n" + summary_obj.summarizer)
        if summary_obj.final_answer:
            parts.append("### RESPUESTA FINAL\n" + summary_obj.final_answer)

        return "\n\n".join(parts)

    # ------------------------------------------------------------------
    # API público
    # ------------------------------------------------------------------
    def run_turn(
        self,
        user_input: Union[str, Dict[str, Any], AgentInput],
    ) -> Dict[str, Any]:
        """
        Ejecuta un turno de conversación.

        Devuelve SIEMPRE un dict:

            {
              "dev_out": {
                  "final_answer": "...pipeline markdown (traza completa)...",
                  "summary": {...},
                  "tool_runs": [...],
                  "raw_state": {...}   # estado crudo del grafo
              },
              "deep_out": {
                  "final_answer": "...resumen por sección...",
                  "summary": {...},
                  "tool_runs": [...],
                  "raw_state": {}      # vista ligera
              },
              "user_out": {
                  "final_answer": "...respuesta para usuario (1:1)...",
                  "summary": {...},
                  "tool_runs": [...],
                  "raw_state": {}
              }
            }
        """
        agent_in = self._coerce_input(user_input)

        # Texto canónico del prompt (user_prompt > user_text como fallback)
        prompt_text = (
            getattr(agent_in, "user_prompt", None)
            or getattr(agent_in, "user_text", None)
            or ""
        )

        # -------------------------
        # 0) Resolver session_id / user_id / kb_names
        # -------------------------
        session_id = agent_in.session_id or "default"
        user_id = None
        if agent_in.metadata:
            user_id = agent_in.metadata.get("user_id")

        kb_names = agent_in.kb_names or []

        # Seleccionar KBs activas para este turno (si kb_names está vacío, usamos todas)
        kb_selected = get_kb_by_names(kb_names, self.knowledge_bases)

        # -------------------------
        # 1) Leer memoria para esta sesión
        # -------------------------
        memory_context = read_memory(session_id=session_id)

        # 2) Construir estado de entrada al grafo
        prev_messages = self._clean_prev_messages()
        state_in: State = {
            "messages": prev_messages + [HumanMessage(content=prompt_text)],
            "analyzer": None,
            "planner_trajs": [],
            "executor_steps": [],
            "tool_runs": [],
            "summary": None,
            "pipeline_summary": None,
            # Campos extra (no tipados en State, pero soportados por TypedDict total=False)
            "user_prompt": prompt_text,
            "session_id": session_id,
            "user_id": user_id,
            "setup_path": self.setup_path or "",
            "setup_config": self.setup_config,  # 👈 YAML completo (por si lo necesita el grafo)
            "kb_names": kb_names,
            "kb_all": [kb.__dict__ for kb in self.knowledge_bases],
            "kb_selected": [kb.__dict__ for kb in kb_selected],
            "memory_context": memory_context,
            # Tablas de contexto (para semantic_search_in_csv en el grafo)
            "context_tables": list(self.context_tables),
            "context_cfg": self.context_cfg,
        }

        # 3) Invocar grafo
        out_state: State = self.graph_app.invoke(state_in)
        self._state = out_state  # guardamos para multi-turn

        # 4) Extraer último AIMessage (por si no vienen campos dev_out/deep_out en state)
        ai_messages = [
            m for m in out_state.get("messages", []) if isinstance(m, AIMessage)
        ]
        last_ai = ai_messages[-1] if ai_messages else None
        last_ai_text = last_ai.content if last_ai is not None else ""

        # 5) Campos de texto que el grafo ya pudo haber dejado en el estado
        dev_text_state = out_state.get("dev_out")  # type: ignore[assignment]
        deep_text_state = out_state.get("deep_out")  # type: ignore[assignment]
        user_text_state = out_state.get("user_out")  # type: ignore[assignment]

        # 6) Summary estructurado (con final_answer "para usuario")
        raw_summary: Dict[str, Any] = (
            out_state.get("pipeline_summary")  # type: ignore[arg-type]
            or out_state.get("summary")  # type: ignore[arg-type]
            or {}
        )
        if raw_summary:
            summary_obj = AgentSummary(**raw_summary)
            summary_user_answer = summary_obj.final_answer or ""
        else:
            summary_obj = None
            summary_user_answer = ""

        # 7) Tool runs normalizados (para las tres vistas)
        raw_runs = out_state.get("tool_runs", []) or []
        tool_runs: List[ToolRun] = []
        for r in raw_runs:
            tool_runs.append(
                ToolRun(
                    id=str(r.get("id", "")),
                    name=str(r.get("name", "")),
                    args=r.get("args", {}),
                    output=r.get("output"),
                )
            )

        # 8) Resolver textos finales por vista con prioridades claras
        # USER:
        #   1) user_text_state (si el grafo lo puso)
        #   2) summary_user_answer (si existe en AgentSummary)
        #   3) last_ai_text (último mensaje de la traza)
        final_user = (
            (user_text_state or "").strip()
            or summary_user_answer.strip()
            or last_ai_text.strip()
        )

        # DEEP:
        #   1) deep_text_state (del grafo)
        #   2) _build_deep_text(summary_obj)
        #   3) summary_user_answer
        final_deep = (
            (deep_text_state or "").strip()
            or self._build_deep_text(summary_obj).strip()
            or summary_user_answer.strip()
            or last_ai_text.strip()
        )

        # DEV:
        #   1) dev_text_state (del grafo: traza markdown completa)
        #   2) last_ai_text
        #   3) deep_text (como fallback)
        final_dev = (
            (dev_text_state or "").strip()
            or last_ai_text.strip()
            or final_deep
        )

        # 9) Construir vistas
        dev_view = AgentView(
            final_answer=final_dev,
            summary=summary_obj,
            tool_runs=tool_runs,
            raw_state=out_state,  # vista completa
        )

        deep_view = AgentView(
            final_answer=final_deep,
            summary=summary_obj,
            tool_runs=tool_runs,
            raw_state={},  # light view
        )

        user_view = AgentView(
            final_answer=final_user,
            summary=summary_obj,
            tool_runs=tool_runs,
            raw_state={},  # usuario nunca necesita estado crudo
        )

        # 10) Actualizar memoria de largo/corto plazo
        try:
            write_memory(
                session_id=session_id,
                user_prompt=prompt_text,
                user_out=final_user,
                user_id=user_id,
                memory_cfg=self.memory_cfg,
            )
        except Exception as e:
            # No queremos que un fallo en memoria rompa el turno
            print(f"[Agent] ⚠️ Error escribiendo memoria: {e!r}")

        # 11) Empaquetar en AgentOutput y devolver como dict puro
        output = AgentOutput(
            dev_out=dev_view,
            deep_out=deep_view,
            user_out=user_view,
        )
        return output.to_dict()

    # ------------------------------------------------------------------
    # Extras útiles para debugging
    # ------------------------------------------------------------------
    @property
    def last_state(self) -> Optional[Dict[str, Any]]:
        """Devuelve el último estado crudo del grafo (solo lectura)."""
        return self._state

    def reset_session(self) -> None:
        """Resetea el estado interno de conversación (no borra memoria global)."""
        self._state = None


In [ ]:
#@title agnostic_agent/__init__.py
%%writefile agnostic_agent/__init__.py
from __future__ import annotations

"""
Agnostic Deep Agent 2026 – paquete principal.

Arquitectura por módulos:

- setup.yaml        → configuración declarativa (modelos, memoria, planner, KBs, etc.).
- schemas.py        → contratos de datos internos (AnalyzerIntent, PlannerPlan, ToolRun…).
- capabilities.py   → capacidades de exploración de entornos:
                      · gestión de modelos / backends (vLLM, OpenAI, etc.)
                      · lanzamiento de servidores Qwen3+vLLM
                      · configuración del planner de herramientas.
- tools.py          → catálogo y registro de tools (toy, matemáticas, embeddings,
                      reranker, tools de contexto/tablas…).
- memory.py         → memoria de sesión / corto / largo plazo.
- prompts.py        → prompts por rol (analyzer, summarizer, validator, memory_write…).
- logic.py          → grafo maestro + sub-grafos
                      (ANALYZER, PLANNER, EXECUTOR, CATCHER, SUMMARIZER, VALIDATOR).
- agent.py          → clase Agent de alto nivel (init, run_turn).
- communication.py  → normalización de I/O (AgentInput, AgentOutput, vistas user/deep/dev).
- context.py        → definición de Knowledge Bases y conectores externos
                      (por ejemplo:
                       · BD tabular en SQLite
                       · VDB en sqlite-vec
                       · otras fuentes RAG, APIs, SQL, etc.).
"""

from .agent import Agent
from .communication import AgentInput, AgentOutput
from .capabilities import (
    PlannerConfig,
    QwenModelPaths,
    VllmConfig,
    VllmServers,
    VllmEndpoints,
    prepare_qwen_models,
    start_qwen_vllm_servers,
)
from .tools import get_default_tools
from .context import KnowledgeBase, get_default_context

__version__ = "0.2.0"

# Alias de compatibilidad con versiones anteriores (legacy)
AgentSession = Agent  # type: ignore

# API pública principal
__all__ = [
    # Núcleo del agente
    "Agent",
    "AgentSession",
    "AgentInput",
    "AgentOutput",
    # Configuración / modelos
    "PlannerConfig",
    "QwenModelPaths",
    "VllmConfig",
    "VllmServers",
    "VllmEndpoints",
    "prepare_qwen_models",
    "start_qwen_vllm_servers",
    # Contexto / tools de alto nivel
    "KnowledgeBase",
    "get_default_tools",
    "get_default_context",
    # Meta
    "__version__",
]


# DEFINE vLLM

In [ ]:
# TRY

#@title DOWNLOAD MODELS – LLM, Embedding, Reranker (Qwen3)
from agnostic_agent import prepare_qwen_models

# IDs de modelos (puedes ajustarlos)
LLM_MODEL_ID = "Qwen/Qwen3-0.6B"  #@param ["Qwen/Qwen3-0.6B", "Qwen/Qwen3-0.6B-Base", "Qwen/Qwen3-4B", "Qwen/Qwen3-4B-Instruct-2507", "Qwen/Qwen3-4B-Thinking-2507"]

EMB_MODEL_ID = "Qwen/Qwen3-Embedding-0.6B"  #@param ["Qwen/Qwen3-Embedding-0.6B"]

RERANK_MODEL_ID = "Qwen/Qwen3-Reranker-0.6B"  #@param ["Qwen/Qwen3-Reranker-0.6B"]

print("⬇️ Descargando / preparando modelos Qwen3 (prepare_qwen_models)...")
model_paths = prepare_qwen_models(
    llm_model_id=LLM_MODEL_ID,
    emb_model_id=EMB_MODEL_ID,
    rerank_model_id=RERANK_MODEL_ID,
    base_dir="LM_MODEL",
)
print("\n✅ Modelos listos en disco:")
print("  LLM   :", model_paths.llm_dir)
print("  EMB   :", model_paths.emb_dir)
print("  RERANK:", model_paths.rerank_dir)

import os
os.environ["LLM_MODEL_ID"]   = LLM_MODEL_ID
os.environ["EMB_MODEL_ID"]   = EMB_MODEL_ID
os.environ["RERANK_MODEL_ID"] = RERANK_MODEL_ID


In [ ]:
#@title LIMPIEZA LIGERA DE GPU (opcional)
import gc
try:
    import torch
    has_torch = True
except ImportError:
    has_torch = False

print("🧹 Limpiando referencias de Python...")
gc.collect()
if has_torch and torch.cuda.is_available():
    print("🧠 Vaciando caché CUDA...")
    torch.cuda.empty_cache()
else:
    print("ℹ️ Torch CUDA no disponible o sin GPU visible.")


In [ ]:
#@title DEPLOY vLLM SERVER – SOLO LLM (chat) · L4/T4-friendly
import transformers
from agnostic_agent import VllmConfig, start_qwen_vllm_servers

print("Transformers version:", transformers.__version__)

VLLM_HOST = os.getenv("VLLM_HOST", "127.0.0.1")
LLM_PORT = 8000
EMB_PORT = 8001
RERANK_PORT = 8002
VLLM_MODE = "POWER"  # "FAST", "MEDIUM", "POWER", "LIMIT"

print(f"\n🚀 Lanzando servidor vLLM (start_qwen_vllm_servers)...")
print(f"🔧 VLLM_MODE = {VLLM_MODE}")

if VLLM_MODE == "FAST":
    vllm_cfg = VllmConfig(
        host=VLLM_HOST,
        llm_port=LLM_PORT,
        emb_port=EMB_PORT,
        rerank_port=RERANK_PORT,
        llm_gpu_util=0.40,
        llm_max_len=2048,
        llm_max_num_seqs=4,
        emb_gpu_util=0.30,
        emb_max_len=512,
        emb_max_num_seqs=4,
        rerank_gpu_util=0.30,
        rerank_max_len=512,
        rerank_max_num_seqs=4,
        enable_reasoning=True,
        tool_call_parser="hermes",
        reasoning_parser="qwen3",
        start_emb_server=False,
        start_rerank_server=False,
    )
elif VLLM_MODE == "MEDIUM":
    vllm_cfg = VllmConfig(
        host=VLLM_HOST,
        llm_port=LLM_PORT,
        emb_port=EMB_PORT,
        rerank_port=RERANK_PORT,
        llm_gpu_util=0.55,
        llm_max_len=4096,
        llm_max_num_seqs=6,
        emb_gpu_util=0.30,
        emb_max_len=512,
        emb_max_num_seqs=4,
        rerank_gpu_util=0.30,
        rerank_max_len=512,
        rerank_max_num_seqs=4,
        enable_reasoning=True,
        tool_call_parser="hermes",
        reasoning_parser="qwen3",
        start_emb_server=False,
        start_rerank_server=False,
    )
elif VLLM_MODE == "POWER":
    # Modo POWER “cargado” para L4 + Qwen3-0.6B:
    # - Más contexto (16k tokens)
    # - Menos concurrencia (4 secuencias)
    # - GPU util ~0.8 para exprimir la L4 sin matarla
    vllm_cfg = VllmConfig(
        # config base
        host=VLLM_HOST,
        llm_port=LLM_PORT,
        emb_port=EMB_PORT,
        rerank_port=RERANK_PORT,
        # language model
        llm_gpu_util=0.90,   # antes 0.70
        llm_max_len=32768,   # antes 8192  ← clave
        llm_max_num_seqs=1,  # antes 8     ← menos batch, más contexto
        # embedding model
        emb_gpu_util=0.05,
        emb_max_len=1024,
        emb_max_num_seqs=1,
        # reranker model
        rerank_gpu_util=0.05,
        rerank_max_len=1024,
        rerank_max_num_seqs=1,
        # config extra
        enable_reasoning=True,
        tool_call_parser="hermes",
        reasoning_parser="qwen3",
        start_emb_server=False,
        start_rerank_server=False,
    )

elif "LIMIT":
    vllm_cfg = VllmConfig(
        host=VLLM_HOST,
        llm_port=LLM_PORT,
        emb_port=EMB_PORT,
        rerank_port=RERANK_PORT,
        llm_gpu_util=0.85,    # un poco más agresivo
        llm_max_len=32768,    # 🔥 32k tokens de contexto
        llm_max_num_seqs=2,   # menos batch, más contexto por secuencia
        emb_gpu_util=0.35,
        emb_max_len=1024,
        emb_max_num_seqs=4,
        rerank_gpu_util=0.35,
        rerank_max_len=1024,
        rerank_max_num_seqs=4,
        enable_reasoning=True, # True | False
        tool_call_parser="hermes",
        reasoning_parser="qwen3",    # "qwen3" | none
        start_emb_server=True,
        start_rerank_server=False,
    )

endpoints, servers = start_qwen_vllm_servers(
    model_paths=model_paths,
    config=vllm_cfg,
    set_env=True,
)

print("\n✅ Servidor vLLM (LLM) listo.")


# RUN EXPERIMENT

In [ ]:
#@title streamlit_app.py
%%writefile streamlit_app.py
from __future__ import annotations

import os
import json
import html
from typing import Any, Dict, Optional, List, Tuple

import streamlit as st
from agnostic_agent.agent import Agent

# -----------------------------
# Page
# -----------------------------
st.set_page_config(
    page_title="Agnostic Agent · Chat Studio (Inspector)",
    page_icon="🧪",
    layout="wide",
)

# -----------------------------
# Hide Streamlit chrome (dark cintillo)
# -----------------------------
st.markdown("""
<style>
[data-testid="stHeader"] { display: none !important; }
[data-testid="stToolbar"] { display: none !important; }
[data-testid="stDecoration"] { display: none !important; }
#MainMenu { visibility: hidden !important; }
footer { visibility: hidden !important; }
.block-container { padding-top: 1rem !important; }
</style>
""", unsafe_allow_html=True)

# -----------------------------
# CSS (Studio + Inspector layout)
# -----------------------------
st.markdown(
    """
<style>
:root{
  --bg: #0b1020;
  --panel: rgba(255,255,255,.06);
  --panel2: rgba(255,255,255,.08);
  --border: rgba(255,255,255,.10);
  --text: rgba(255,255,255,.92);
  --muted: rgba(255,255,255,.65);
  --accent: #7c5cff;
  --good: #2dd4bf;

  --r: 18px;
  --r2: 14px;
  --shadow: 0 12px 35px rgba(0,0,0,.35);
  --mono: ui-monospace, SFMono-Regular, Menlo, Monaco, Consolas, "Liberation Mono", "Courier New", monospace;
}

.stApp{
  background:
    radial-gradient(1200px 500px at 10% -10%, rgba(124,92,255,.35), transparent 60%),
    radial-gradient(900px 500px at 90% 0%, rgba(45,212,191,.18), transparent 60%),
    linear-gradient(180deg, var(--bg), #070a14 60%, #050712);
  color: var(--text);
}

.block-container{ padding-top: 1.0rem; padding-bottom: 1.6rem; }

section[data-testid="stSidebar"]{
  background: rgba(0,0,0,.18);
  border-right: 1px solid rgba(255,255,255,.06);
}

.topbar{
  display:flex; align-items:center; justify-content:space-between;
  gap:12px;
  padding: 12px 14px;
  border-radius: var(--r);
  background: linear-gradient(180deg, rgba(255,255,255,.08), rgba(255,255,255,.05));
  border: 1px solid var(--border);
  box-shadow: var(--shadow);
  margin-bottom: 10px;
}
.brand{display:flex; align-items:center; gap:10px;}
.logo{
  width: 38px; height: 38px; border-radius: 12px;
  display:flex; align-items:center; justify-content:center;
  background: linear-gradient(135deg, rgba(124,92,255,.9), rgba(45,212,191,.6));
  box-shadow: 0 10px 25px rgba(124,92,255,.22);
  font-size: 18px;
}
.title{font-size: 15px; font-weight: 800; line-height: 1.1;}
.subtitle{font-size: 12px; color: var(--muted);}

.badges{display:flex; flex-wrap:wrap; gap:8px; justify-content:flex-end;}
.badge{
  font-size: 12px;
  padding: 6px 10px;
  border-radius: 999px;
  border: 1px solid var(--border);
  background: rgba(255,255,255,.06);
  color: var(--text);
}
.badge.accent{ border-color: rgba(124,92,255,.45); }
.badge.good{ border-color: rgba(45,212,191,.45); }

.card{
  border-radius: var(--r);
  border: 1px solid var(--border);
  background: rgba(255,255,255,.06);
  box-shadow: 0 10px 30px rgba(0,0,0,.28);
  overflow: hidden;
  margin-top: 6px;
}
.card .card-h{
  display:flex; align-items:center; justify-content:space-between;
  gap:10px;
  padding: 10px 12px;
  background: rgba(255,255,255,.05);
  border-bottom: 1px solid var(--border);
  font-weight: 800;
  font-size: 13px;
}
.card .card-h .hint{
  font-weight: 500; font-size: 11px; color: var(--muted);
}
.card .card-b{
  padding: 12px 12px 10px 12px;
  font-size: 14px;
  color: var(--text);
}

/* Code-like block inside cards (for Thinking) */
.codebox{
  margin-top: 8px;
  padding: 10px 12px;
  border-radius: 14px;
  border: 1px solid rgba(255,255,255,.10);
  background: rgba(0,0,0,.28);
  font-family: var(--mono);
  font-size: 12px;
  line-height: 1.45;
  white-space: pre-wrap;
  word-break: break-word;
  color: rgba(255,255,255,.92);
}

/* User bubble */
.bubble-user{
  padding: 10px 12px;
  border-radius: 16px;
  border: 1px solid rgba(124,92,255,.35);
  background: linear-gradient(180deg, rgba(124,92,255,.22), rgba(255,255,255,.05));
  box-shadow: 0 8px 24px rgba(0,0,0,.25);
}

/* Inspector wrapper */
.inspector{
  border-radius: var(--r);
  border: 1px solid var(--border);
  background: rgba(255,255,255,.05);
  box-shadow: var(--shadow);
  padding: 12px;
}
.inspector h3{ margin: 0 0 6px 0; }

/* Expanders */
[data-testid="stExpander"]{
  border-radius: var(--r);
  border: 1px solid var(--border);
  background: rgba(255,255,255,.04);
  overflow:hidden;
}

/* Chat spacing */
[data-testid="stChatMessage"]{
  padding-top: 0.25rem;
  padding-bottom: 0.25rem;
}
</style>
""",
    unsafe_allow_html=True,
)

# -----------------------------
# Session state
# -----------------------------
if "agent" not in st.session_state:
    st.session_state.agent = None
if "agent_mode" not in st.session_state:
    st.session_state.agent_mode = "tools_strict"
if "messages" not in st.session_state:
    st.session_state.messages = []
if "msg_counter" not in st.session_state:
    st.session_state.msg_counter = 0
if "selected_msg_id" not in st.session_state:
    st.session_state.selected_msg_id = None
if "export_json" not in st.session_state:
    st.session_state.export_json = None

# -----------------------------
# Sidebar controls
# -----------------------------
with st.sidebar:
    st.markdown("## 🧪 Chat Studio")
    agent_mode = st.selectbox(
        "Policy mode",
        ["tools_strict", "free_policies"],
        index=0 if st.session_state.agent_mode == "tools_strict" else 1,
    )

    st.markdown("### 🧭 Inspector")
    show_thinking_tab = st.checkbox("🧠 Thinking", value=True)
    show_deep_tab = st.checkbox("🧠 Deep", value=True)
    show_dev_tab = st.checkbox("🔍 Dev", value=True)

    st.markdown("### 🧹 Acciones")
    cA, cB = st.columns(2)
    with cA:
        if st.button("🗑️ Limpiar", use_container_width=True):
            st.session_state.messages = []
            st.session_state.agent = None
            st.session_state.selected_msg_id = None
            st.toast("Chat reiniciado.", icon="🧹")
            st.rerun()
    with cB:
        if st.button("⬇️ Export", use_container_width=True):
            export = {"agent_mode": st.session_state.agent_mode, "messages": st.session_state.messages}
            st.session_state.export_json = json.dumps(export, ensure_ascii=False, indent=2)
            st.toast("Transcript listo.", icon="⬇️")

    if isinstance(st.session_state.export_json, str):
        st.download_button(
            "Descargar transcript.json",
            data=st.session_state.export_json,
            file_name="transcript.json",
            mime="application/json",
            use_container_width=True,
        )

# Mode change => reset agent (history stays)
if st.session_state.agent_mode != agent_mode:
    st.session_state.agent_mode = agent_mode
    st.session_state.agent = None
    st.toast(f"Modo: {agent_mode}", icon="🧭")

# -----------------------------
# Helpers
# -----------------------------
def next_id() -> int:
    st.session_state.msg_counter += 1
    return st.session_state.msg_counter

def get_or_init_agent(mode: str) -> Agent:
    if st.session_state.agent is None:
        os.environ["PLANNER_POLICY_MODE"] = mode
        os.environ["AGENT_POLICY_MODE"] = mode
        with st.spinner(f"Inicializando agente (mode={mode})…"):
            try:
                st.session_state.agent = Agent.init(policy_mode=mode)
            except TypeError:
                st.session_state.agent = Agent.init()
    return st.session_state.agent

def normalize_output(raw: Any) -> Dict[str, Any]:
    if raw is None:
        return {}
    if hasattr(raw, "model_dump"):
        try:
            return raw.model_dump()
        except TypeError:
            pass
    if isinstance(raw, dict):
        return raw
    return {"user_out": str(raw)}

def as_text(v: Any) -> str:
    if v is None:
        return ""
    if isinstance(v, str):
        return v.strip()
    if isinstance(v, dict):
        for k in ("final_answer", "text", "content", "answer", "user_out"):
            vv = v.get(k)
            if isinstance(vv, str) and vv.strip():
                return vv.strip()
        return ""
    return str(v).strip()

def strip_user_prefix(text: str) -> str:
    if not text:
        return ""
    t = text.strip()
    prefixes = [
        "Respuesta final (modo usuario):",
        "**Respuesta final (modo usuario):**",
        "RESPUESTA FINAL (modo usuario):",
    ]
    for p in prefixes:
        if t.startswith(p):
            t = t[len(p):].strip()
    return t

def get_raw_state(out: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    if not isinstance(out, dict):
        return None
    if isinstance(out.get("messages"), list):
        return out
    rs = out.get("raw_state")
    if isinstance(rs, dict):
        return rs
    dev = out.get("dev_out")
    if isinstance(dev, dict) and isinstance(dev.get("raw_state"), dict):
        return dev["raw_state"]
    return None

def extract_thinking(raw_state: Optional[Dict[str, Any]]) -> str:
    if not isinstance(raw_state, dict):
        return ""
    msgs = raw_state.get("messages")
    if not isinstance(msgs, list):
        return ""
    for m in reversed(msgs):
        if not isinstance(m, dict):
            continue
        if m.get("type") != "ai":
            continue
        ak = m.get("additional_kwargs") or {}
        if isinstance(ak, dict) and ak.get("pipeline_internal"):
            continue
        thinking = ak.get("reasoning_content") or ak.get("reasoning") or ak.get("thoughts") or ""
        return thinking.strip() if isinstance(thinking, str) else ""
    return ""

def extract_summary_deep(raw_state: Optional[Dict[str, Any]], deep_out_text: str) -> str:
    if deep_out_text:
        return deep_out_text
    if not isinstance(raw_state, dict):
        return ""
    summary = raw_state.get("summary") or raw_state.get("pipeline_summary")
    if not isinstance(summary, dict):
        return ""
    parts = []
    for k in ["analyzer", "planner", "executor", "catcher", "summarizer", "final_answer"]:
        v = summary.get(k, "")
        if isinstance(v, str) and v.strip():
            parts.append(f"**{k.upper()}**\n\n{v.strip()}")
    return "\n\n---\n\n".join(parts) if parts else ""

def extract_tool_runs(out: Dict[str, Any], raw_state: Optional[Dict[str, Any]]) -> List[Dict[str, Any]]:
    if isinstance(raw_state, dict):
        tr = raw_state.get("tool_runs")
        if isinstance(tr, list):
            return tr
    tr2 = out.get("tool_runs")
    if isinstance(tr2, list):
        return tr2
    dev = out.get("dev_out")
    if isinstance(dev, dict) and isinstance(dev.get("tool_runs"), list):
        return dev["tool_runs"]
    return []

def assistant_messages() -> List[Dict[str, Any]]:
    return [m for m in st.session_state.messages if m.get("role") == "assistant"]

def find_message_by_id(msg_id: Optional[int]) -> Optional[Dict[str, Any]]:
    if msg_id is None:
        return None
    for m in st.session_state.messages:
        if m.get("id") == msg_id:
            return m
    return None

def default_selected_id() -> Optional[int]:
    a = assistant_messages()
    return a[-1]["id"] if a else None

def card_md(title: str, body_md: str, icon: str = "⬛", hint: str = "") -> None:
    body_md = body_md or "_(vacío)_"
    hint_html = f'<span class="hint">{html.escape(hint)}</span>' if hint else ""
    # NOTE: body_md here is treated as plain HTML content; for Deep this is OK.
    # For Thinking we use code-card below so it looks like "markdown blocks".
    st.markdown(
        f"""
<div class="card">
  <div class="card-h">
    <div>{icon} {html.escape(title)}</div>
    {hint_html}
  </div>
  <div class="card-b">{body_md}</div>
</div>
""",
        unsafe_allow_html=True,
    )

def card_code(title: str, code_text: str, icon: str = "🧠", hint: str = "reasoning_content") -> None:
    safe = html.escape(code_text or "")
    hint_html = f'<span class="hint">{html.escape(hint)}</span>' if hint else ""
    content = safe if safe.strip() else html.escape("_(no viene thinking en este turno)_")
    st.markdown(
        f"""
<div class="card">
  <div class="card-h">
    <div>{icon} {html.escape(title)}</div>
    {hint_html}
  </div>
  <div class="card-b">
    <div class="codebox">{content}</div>
  </div>
</div>
""",
        unsafe_allow_html=True,
    )

def render_tool_runs(tool_runs: List[Dict[str, Any]]) -> None:
    if not tool_runs:
        st.markdown("_(No se ejecutaron tools en este turno.)_")
        return
    st.markdown("#### 🛠 Tools ejecutadas")
    for i, tr in enumerate(tool_runs, start=1):
        if not isinstance(tr, dict):
            st.markdown(f"**{i}. tool_{i}**")
            st.code(str(tr))
            continue
        name = tr.get("name", f"tool_{i}")
        args = tr.get("args", {})
        output = tr.get("output", "")
        st.markdown(f"**{i}. {name}**")
        if args:
            st.code(args, language="json")
        if output != "":
            st.markdown("**Salida:**")
            st.code(str(output))

# If nothing selected yet, default to last assistant
if st.session_state.selected_msg_id is None:
    st.session_state.selected_msg_id = default_selected_id()

# -----------------------------
# Top bar
# -----------------------------
mode_badge = "tools_strict" if st.session_state.agent_mode == "tools_strict" else "free_policies"
mode_class = "accent" if st.session_state.agent_mode == "tools_strict" else "good"

st.markdown(
    f"""
<div class="topbar">
  <div class="brand">
    <div class="logo">🧪</div>
    <div>
      <div class="title">Agnostic Agent · Chat Studio</div>
      <div class="subtitle">Feed limpio + Inspector dinámico (thinking/deep/dev)</div>
    </div>
  </div>
  <div class="badges">
    <span class="badge {mode_class}">🧭 {mode_badge}</span>
    <span class="badge">🔎 inspector: on</span>
  </div>
</div>
""",
    unsafe_allow_html=True,
)

# -----------------------------
# Main layout: Feed | Inspector
# -----------------------------
feed_col, insp_col = st.columns([2.2, 1.0], gap="large")

# -------- FEED (left) --------
with feed_col:
    for msg in st.session_state.messages:
        role = msg.get("role", "user")

        if role == "user":
            with st.chat_message("user"):
                st.markdown(f'<div class="bubble-user">{html.escape(msg.get("content",""))}</div>', unsafe_allow_html=True)

        elif role == "assistant":
            out = msg.get("out") or {}
            answer = strip_user_prefix(as_text(out.get("user_out")))
            raw_state = get_raw_state(out)
            tool_runs = extract_tool_runs(out, raw_state)

            with st.chat_message("assistant"):
                # Pretty answer only
                card_md(
                    title="Respuesta",
                    body_md=html.escape(answer or "_(sin respuesta)_").replace("\n", "<br>"),
                    icon="👤",
                    hint=f"id={msg.get('id')}",
                )

                c1, c2, c3 = st.columns([1.2, 1.0, 0.8])
                with c1:
                    st.caption(f"🛠 tools: {len(tool_runs)}")
                with c2:
                    st.caption("📎 Inspector →")
                with c3:
                    if st.button("🔎 Inspect", key=f"inspect_{msg.get('id')}", use_container_width=True):
                        st.session_state.selected_msg_id = msg.get("id")
                        st.toast(f"Inspector → id={msg.get('id')}", icon="🔎")
                        st.rerun()

# -------- INSPECTOR (right) --------
with insp_col:
    st.markdown('<div class="inspector">', unsafe_allow_html=True)
    st.markdown("### 🔎 Inspector")

    a_msgs = assistant_messages()
    if not a_msgs:
        st.info("Aún no hay respuestas del agente. Escribe algo para empezar.")
        st.markdown("</div>", unsafe_allow_html=True)
    else:
        ids = [m["id"] for m in a_msgs]

        def label(mid: int) -> str:
            m = find_message_by_id(mid) or {}
            out = m.get("out") or {}
            text = strip_user_prefix(as_text(out.get("user_out"))).replace("\n", " ").strip()
            text = (text[:60] + "…") if len(text) > 60 else text
            return f"id={mid} · {text or '(sin texto)'}"

        if st.session_state.selected_msg_id not in ids:
            st.session_state.selected_msg_id = ids[-1]

        sel = st.selectbox(
            "Mensaje seleccionado",
            options=ids,
            index=ids.index(st.session_state.selected_msg_id),
            format_func=label,
            key="inspector_selectbox",
        )
        st.session_state.selected_msg_id = sel

        m = find_message_by_id(st.session_state.selected_msg_id) or {}
        out = m.get("out") or {}
        raw_state = get_raw_state(out)

        thinking = extract_thinking(raw_state)
        deep_txt = extract_summary_deep(raw_state, as_text(out.get("deep_out")))
        tool_runs = extract_tool_runs(out, raw_state)

        tab_specs: List[Tuple[str, str]] = []
        if show_thinking_tab:
            tab_specs.append(("🧠 Thinking", "thinking"))
        if show_deep_tab:
            tab_specs.append(("🧠 Deep", "deep"))
        if show_dev_tab:
            tab_specs.append(("🔍 Dev", "dev"))

        tabs = st.tabs([t[0] for t in tab_specs])

        for (tab_title, tab_key), tab in zip(tab_specs, tabs):
            with tab:
                if tab_key == "thinking":
                    # ✅ Now it looks like a proper "markdown code block" card (monospace, wrapped, bordered)
                    card_code("Pensamiento (thinking)", thinking, icon="🧠", hint="reasoning_content")

                elif tab_key == "deep":
                    # Keep as-is (you said you like it)
                    # We keep newlines readable and let emphasis render in markdown-ish style
                    body = deep_txt or "_(vacío)_"
                    # Render markdown emphasis by replacing to HTML-ish layout; deep usually ok as plain text too
                    body_html = html.escape(body).replace("\n", "<br>")
                    card_md("Vista profunda (deep_out / summary)", body_html, icon="🧠", hint="pipeline")

                elif tab_key == "dev":
                    # ✅ Dev: tools like before + raw_state collapsed
                    render_tool_runs(tool_runs)

                    with st.expander("🧬 raw_state (debug)", expanded=False):
                        if isinstance(raw_state, dict) and raw_state:
                            st.json(raw_state)
                        else:
                            st.markdown("_(sin raw_state)_")

        st.markdown("</div>", unsafe_allow_html=True)

# -----------------------------
# Chat input (bottom)
# -----------------------------
prompt = st.chat_input("Escribe tu mensaje…")

if prompt:
    uid = next_id()
    st.session_state.messages.append({"id": uid, "role": "user", "content": prompt, "out": None})

    agent = get_or_init_agent(st.session_state.agent_mode)
    try:
        raw_out = agent.run_turn(prompt, policy_mode=st.session_state.agent_mode)
    except TypeError:
        raw_out = agent.run_turn(prompt)

    out = normalize_output(raw_out)
    aid = next_id()
    st.session_state.messages.append(
        {"id": aid, "role": "assistant", "content": strip_user_prefix(as_text(out.get("user_out"))), "out": out}
    )

    st.session_state.selected_msg_id = aid
    st.rerun()


In [ ]:
#@title .streamlit/config.toml
!mkdir -p .streamlit

In [ ]:
%%writefile .streamlit/config.toml
[theme]
base="dark"
primaryColor="#7c5cff"
backgroundColor="#070a14"
secondaryBackgroundColor="#10162b"
textColor="#e9e9f2"
font="sans serif"


In [ ]:
#@title Lanzar servicio Streamlit en Colab (window o iframe)
import subprocess, time, os

# ---- PARAMS (Colab UI) ----
PORT = 8501  #@param {type:"integer"}
MODE = "iframe"  #@param ["window", "iframe"]
IFRAME_HEIGHT = 800  #@param {type:"integer"}
IFRAME_WIDTH = "100%"  #@param {type:"string"}

# ---- Colab output (solo si existe) ----
try:
    from google.colab import output
    _IN_COLAB = True
except Exception:
    output = None
    _IN_COLAB = False

# ---- Mata procesos previos (opcional) ----
os.system('pkill -f "streamlit run streamlit_app.py" || true')
os.system('pkill -f "streamlit" || true')

# ---- Lanza Streamlit ----
cmd = [
    "streamlit", "run", "streamlit_app.py",
    "--server.address", "0.0.0.0",
    "--server.port", str(PORT),
    "--server.headless", "true",
    "--server.enableCORS", "false",
    "--server.enableXsrfProtection", "false",
]

streamlit_process = subprocess.Popen(cmd)
time.sleep(3)

# ---- Exponer ----
if _IN_COLAB and output is not None:
    if MODE == "iframe":
        output.serve_kernel_port_as_iframe(
            PORT,
            width=IFRAME_WIDTH,
            height=IFRAME_HEIGHT,
        )
    else:
        output.serve_kernel_port_as_window(
            PORT,
            anchor_text="🔗 Abrir app Streamlit en una nueva pestaña"
        )
else:
    print(f"Streamlit corriendo en http://127.0.0.1:{PORT} (no-Colab).")
